# Gpu

In [1]:
!nvidia-smi

Sat Sep 20 15:42:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Set_Up

In [4]:

symbol = 'XAUUSD'
strategy = 'Kalman'
time_frame = 'M5'

root_data = f'/content/drive/MyDrive/Course Folder/Forex/XAUUSD/'
print(root_data)

rolling_window = 100
lot_size = 100

/content/drive/MyDrive/Course Folder/Forex/XAUUSD/


# Libraries

In [5]:
!pip install ta-lib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 61.2 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
from typing import Sequence, Tuple, Dict, Any, List, Callable
import random


from tqdm.auto import tqdm
from tqdm import tqdm

import sys
sys.path.append("..")

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-v0_8-darkgrid")

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!conda list ta-lib
import talib as ta
print(ta.__version__)
print(help(ta.ATR))

/bin/bash: line 1: conda: command not found
0.6.7
Help on function ATR in module talib._ta_lib:

ATR(high, low, close, timeperiod=-2147483648)
    ATR(ndarray high, ndarray low, ndarray close, int timeperiod=-0x80000000)

    ATR(high, low, close[, timeperiod=?])

    Average True Range (Volatility Indicators)

    Inputs:
        prices: ['high', 'low', 'close']
    Parameters:
        timeperiod: 14
    Outputs:
        real

None


## Strategy

In [10]:
def kalman_line(source, kalman_length: int, smooth: int):

    n = len(source)
    kf_c = np.empty(n)            # núcleo del filtro
    velo_c = np.zeros(n)          # componente de velocidad

    sqrt_term   = np.sqrt(kalman_length / 10000.0 * 2.0)
    length_term = kalman_length / 10000.0

    # --------- inicialización (mismo efecto que `var` en Pine) ----------
    kf_c[0]   = source.iloc[0]    # nz(kf_c[1], source) para la primera barra
    velo_c[0] = 0.0

    # ------------------- bucle recursivo -------------------------------
    for i in range(1, n):
        prev_kf = kf_c[i-1] if not np.isnan(kf_c[i-1]) else source.iloc[i]
        dk      = source.iloc[i] - prev_kf
        smooth_c = prev_kf + dk * sqrt_term          # parte "suave"
        velo_c[i] = velo_c[i-1] + length_term * dk   # acumulamos velocidad
        kf_c[i]   = smooth_c + velo_c[i]             # estimación final

    # -------------------- EMA final (ta.ema) ----------------------------
    kf_c_series = pd.Series(kf_c, index=source.index)
    kalman_line = kf_c_series.ewm(span=smooth, adjust=False).mean()
    return kalman_line


In [11]:
def insta(src: Sequence[float], a: float) -> Tuple[np.ndarray, np.ndarray]:

    src = np.asarray(src, dtype=float)
    n = src.size
    it = np.full(n, np.nan)
    lag = np.full(n, np.nan)

    def get(arr, i):
        return arr[i] if 0 <= i < n else np.nan

    def nz(x, y):
        return x if not np.isnan(x) else y

    for i in range(n):
        s0 = get(src, i)
        s1 = get(src, i - 1)
        s2 = get(src, i - 2)

        # Fallback used inside nz(..., fallback)
        fallback = (s0 + 2 * s1 + s2) / 4 if not (
            np.isnan(s0) or np.isnan(s1) or np.isnan(s2)
        ) else np.nan

        it_prev1 = get(it, i - 1)
        it_prev2 = get(it, i - 2)

        term1 = (a - (a * a) / 4.0) * s0
        term2 = 0.5 * a * a * s1 if not np.isnan(s1) else 0.0
        term3 = -(a - 0.75 * a * a) * s2 if not np.isnan(s2) else 0.0
        term4 = 2 * (1 - a) * nz(it_prev1, fallback)
        term5 = (1 - a) * (1 - a) * nz(it_prev2, fallback)

        # Full recursive update (mirrors Pine order)
        value_it = term1 + term2 + term3 + term4 - term5
        it[i] = value_it

        # lag = 2 * it - nz(it[2])
        it_prev2_for_lag = get(it, i - 2)
        lag[i] = 2 * it[i] - it_prev2_for_lag if not np.isnan(it_prev2_for_lag) else np.nan

    return lag, it

# Data

In [12]:
#/content/drive/MyDrive/Course Folder/Forex/XAUUSD/Data/XAUUSD_M5.csv

data = pd.read_csv(root_data + 'Data/'+symbol+'_'+time_frame+'.csv', index_col=0)
data.index = pd.to_datetime(data.index)
time_difference = data.index.max() - data.index.min()
number_of_days = time_difference.days
data = data.reset_index()
data['ATR'] = ta.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)

print(f"The data DataFrame covers a period of {number_of_days} days.")
print('Shape', data.shape)
data.tail()

The data DataFrame covers a period of 2396 days.
Shape (465718, 8)


,Date,Open,High,Low,Close,Volume,Spread,ATR
465713,2025-07-25 23:35:00,3338.59,3339.14,3338.04,3338.81,197,5,1.135449
465714,2025-07-25 23:40:00,3338.81,3338.92,3337.56,3338.69,194,5,1.151488
465715,2025-07-25 23:45:00,3338.69,3338.69,3336.64,3336.73,195,40,1.215667
465716,2025-07-25 23:50:00,3336.73,3336.86,3336.12,3336.52,192,40,1.181691
465717,2025-07-25 23:55:00,3336.52,3336.88,3336.44,3336.80,82,40,1.128713


In [13]:
#data = data.iloc[0:50000]
data = data.iloc[-20000:]
#data = data.iloc[-500000:]
data.shape

(20000, 8)

In [14]:
# data.head()

# Strategy

In [15]:

length_1 = 300
length_2 = 600
length_3 = 900
length_4 = 1000

smooth_1 = 3
smooth_2 = 3
smooth_3 = 3
smooth_4 = 3

data['kal_1'] = kalman_line(data['Close'],length_1, smooth_1)
data['kal_2'] = kalman_line(data['Close'],length_2, smooth_2)
data['kal_3'] = kalman_line(data['Close'],length_3, smooth_3)
data['kal_4'] = kalman_line(data['Close'],length_4, smooth_4)

In [16]:
bullish_condition = (data['kal_1'] > data['kal_1'].shift()) & \
                    (data['kal_2'] > data['kal_2'].shift()) & \
                    (data['kal_3'] > data['kal_3'].shift())

bearish_condition = (data['kal_1'] < data['kal_1'].shift()) & \
                    (data['kal_2'] < data['kal_2'].shift()) & \
                    (data['kal_3'] < data['kal_3'].shift())


data['Open_Trade_1'] = np.where(bullish_condition, 1, np.where(bearish_condition, -1, np.nan))
data['Open_Trade'] = np.where(data['Open_Trade_1'] == data['Open_Trade_1'].shift(1), np.nan, data['Open_Trade_1'])

close_bearish_condition = data['Open_Trade'] == 1
close_bullish_condition = data['Open_Trade'] == -1

data['Close_Trade_1'] = np.where(data['kal_4'] < data['kal_4'].shift(), -1, np.where(data['kal_4'] > data['kal_4'].shift(), 1, np.nan))
data['Close_Trade'] = np.where(data['Close_Trade_1'] == data['Close_Trade_1'].shift(1), np.nan, data['Close_Trade_1'])

data = data.drop(columns=['Open_Trade_1', 'Close_Trade_1'])
#data

In [17]:
data['Entry_Date'] = pd.NaT
data.loc[data['Open_Trade'] ==  1, 'Entry_Date'] = data['Date'][data['Open_Trade'] == 1]
data.loc[data['Open_Trade'] == -1, 'Entry_Date'] = data['Date'][data['Open_Trade'] == -1]

data['Type'] = np.nan
data.loc[data['Open_Trade'] ==  1, 'Type'] = 'Long'
data.loc[data['Open_Trade'] == -1, 'Type'] = 'Short'
#data.loc[data['Open_Trade'] == -1, 'Type'] = 'Short'
#data['Type'] = data['Type'].ffill()

data['Trade_Number'] = np.nan
start_mask = data["Entry_Date"].notna() & data["Type"].notna()

data["Trade_Number"] = start_mask.cumsum()
first_trade = data["Trade_Number"].ne(0).idxmax()
data.loc[: first_trade - 1, "Trade_Number"] = np.nan
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number'],
      dtype='object')

In [18]:
columns_to_see = ['Date', 'Close','Open_Trade', 'Type', 'Entry_Date','st_Exit_Date','Trade_Number']

data.loc[data['Open_Trade'].notna(), :].head(10)
data.loc[((data['Open_Trade'].notna()) & (data['Type']=='Short')), :].head(10)

,Date,Open,High,Low,Close,Volume,Spread,ATR,kal_1,kal_2,kal_3,kal_4,Open_Trade,Close_Trade,Entry_Date,Type,Trade_Number
445719,2025-04-15 04:25:00,3224.38,3224.45,3222.09,3223.35,643,7,2.405472,3224.203900,3224.138827,3224.085439,3224.069129,-1.0,-1.0,2025-04-15 04:25:00,Short,1.0
445727,2025-04-15 05:05:00,3223.86,3224.26,3220.79,3221.15,494,8,2.707828,3222.470076,3222.325547,3222.357729,3222.379363,-1.0,-1.0,2025-04-15 05:05:00,Short,3.0
445757,2025-04-15 07:35:00,3227.76,3228.37,3227.46,3227.53,303,7,1.981821,3228.917118,3228.700803,3228.496295,3228.435024,-1.0,NaN,2025-04-15 07:35:00,Short,6.0
445764,2025-04-15 08:10:00,3230.24,3230.24,3227.48,3227.77,356,8,2.010218,3231.034705,3230.831159,3230.630425,3230.559244,-1.0,NaN,2025-04-15 08:10:00,Short,8.0
445774,2025-04-15 09:00:00,3224.63,3224.74,3223.23,3223.56,549,5,2.285828,3223.702544,3223.170747,3223.364661,3223.457486,-1.0,NaN,2025-04-15 09:00:00,Short,9.0
445784,2025-04-15 09:50:00,3225.89,3226.93,3224.91,3225.42,436,5,2.454562,3226.788670,3227.373480,3227.283515,3227.207782,-1.0,NaN,2025-04-15 09:50:00,Short,11.0
445809,2025-04-15 11:55:00,3227.99,3228.68,3227.12,3227.73,391,5,2.714855,3229.691683,3229.382528,3228.692471,3228.512061,-1.0,NaN,2025-04-15 11:55:00,Short,13.0
445811,2025-04-15 12:05:00,3229.47,3229.88,3228.16,3228.17,416,6,2.632860,3229.738395,3229.133662,3228.610000,3228.514660,-1.0,-1.0,2025-04-15 12:05:00,Short,14.0
445814,2025-04-15 12:20:00,3229.07,3229.68,3227.06,3227.31,438,5,2.536255,3229.480885,3228.700141,3228.391183,3228.355718,-1.0,-1.0,2025-04-15 12:20:00,Short,16.0
445828,2025-04-15 13:30:00,3224.39,3224.88,3222.53,3222.98,314,5,2.164898,3223.737526,3224.043954,3224.297019,3224.330651,-1.0,NaN,2025-04-15 13:30:00,Short,18.0


# PnL_Original

## Functions

In [19]:
def st_pnl_close(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # 1) ensure Date is datetime & sorted
    if not np.issubdtype(df['Date'].dtype, np.datetime64):
        df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')

    df['st_row_PnL_close'] = np.nan

    # 3) pick only true entry‐rows: longs open_trade==1, shorts open_trade==-1
    entries = df[
        ((df['Type'] == 'Long')  & (df['Open_Trade'] == 1)) |
        ((df['Type'] == 'Short') & (df['Open_Trade'] == -1))
    ]

    for _, trade in entries.iterrows():
        entry    = trade['Entry_Date']
        exit_    = trade['st_Exit_Date'] if pd.notna(trade['st_Exit_Date']) else entry
        base     = trade['Close']
        mask     = (df['Date'] >= entry) & (df['Date'] <= exit_)

        if trade['Type'] == 'Long':
            df.loc[mask, 'st_row_PnL_close'] = (df.loc[mask, 'Close'] - base) * lot_size
        else:
            df.loc[mask, 'st_row_PnL_close'] = (base - df.loc[mask, 'Close']) * lot_size

    return df


def st_pnl_low(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if not np.issubdtype(df['Date'].dtype, np.datetime64):
        df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')

    df['st_row_PnL_Low'] = np.nan

    opens = df[df['Type'].notna()]
    for _, trade in opens.iterrows():
        entry = trade['Entry_Date']
        exit_ = trade['st_Exit_Date']

        # Buscar el precio de cierre en la fecha de entrada
        base_row = df[df['Date'] == entry]
        if base_row.empty:
            continue  # Saltar si no se encuentra esa fecha
        base = base_row.iloc[0]['Close']

        mask = (df['Date'] >= entry) & (df['Date'] <= exit_)

        if trade['Type'] == 'Long':
            df.loc[mask, 'st_row_PnL_low'] = (df.loc[mask, 'Low'] - base) * lot_size
        elif trade['Type'] == 'Short':
            df.loc[mask, 'st_row_PnL_low'] = (base - df.loc[mask, 'Low']) * lot_size

    return df

def st_pnl_high(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if not np.issubdtype(df['Date'].dtype, np.datetime64):
        df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')

    df['st_row_PnL_high'] = np.nan

    opens = df[df['Type'].notna()]
    for _, trade in opens.iterrows():
        entry = trade['Entry_Date']
        exit_ = trade['st_Exit_Date']

        # Buscar el precio de cierre en la fecha de entrada
        base_row = df[df['Date'] == entry]
        if base_row.empty:
            continue  # Saltar si no se encuentra esa fecha
        base = base_row.iloc[0]['Close']

        mask = (df['Date'] >= entry) & (df['Date'] <= exit_)

        if trade['Type'] == 'Long':
            df.loc[mask, 'st_row_PnL_high'] = (df.loc[mask, 'High'] - base) * lot_size
        elif trade['Type'] == 'Short':
            df.loc[mask, 'st_row_PnL_high'] = (base - df.loc[mask, 'High']) * lot_size

    return df

In [20]:
def st_exit_date(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['st_Exit_Date'] = pd.NaT

    long_entries = df[df['Open_Trade'] == 1]
    short_entries = df[df['Open_Trade'] == -1]
    short_signals = df['Date'][df['Open_Trade'] == -1]
    long_signals = df['Date'][df['Open_Trade'] == 1]

    for index, row in long_entries.iterrows():
        next_short_signal_date = short_signals[short_signals > row['Date']]
        if not next_short_signal_date.empty:
            exit_index = df[df['Date'] == next_short_signal_date.iloc[0]].index
            if not exit_index.empty:
                df.loc[index, 'st_Exit_Date'] = df.loc[exit_index[0], 'Date']

    for index, row in short_entries.iterrows():
        next_long_signal_date = long_signals[long_signals > row['Date']]
        if not next_long_signal_date.empty:
            exit_index = df[df['Date'] == next_long_signal_date.iloc[0]].index
            if not exit_index.empty:
                df.loc[index, 'st_Exit_Date'] = df.loc[exit_index[0], 'Date']
    return df

In [21]:
def st_duration(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['st_Duration'] = pd.NaT
    trade_rows = df[df['Type'].isin(['Long', 'Short'])]
    df.loc[trade_rows.index, 'st_Duration'] = (trade_rows['st_Exit_Date'] - trade_rows['Entry_Date']).dt.total_seconds() / 60
    return df

In [22]:
def st_max_min(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df['Entry_Date'] = pd.to_datetime(df['Entry_Date'])
    df['st_Exit_Date']  = pd.to_datetime(df['st_Exit_Date'])

    df['st_Max'] = np.nan
    df['st_Min'] = np.nan

    opens = df[df['Type'].notna()]
    for idx, trade in opens.iterrows():
        entry    = trade['Entry_Date']
        exit_dt  = trade['st_Exit_Date']
        # hallar la fila de exit_dt y retroceder una
        exit_rows = df.index[df['Date'] == exit_dt]
        if not exit_rows.empty and exit_rows[0] > 0:
            prev_date = df.at[exit_rows[0] - 1, 'Date']
        else:
            continue

        mask = (df['Date'] >= entry) & (df['Date'] <= prev_date)
        # Consider max/min across the three specified columns within the mask
        subset_df = df.loc[mask, ['st_row_PnL_close', 'st_row_PnL_low', 'st_row_PnL_high']]

        if not subset_df.empty:
            # Calculate max and min across the relevant columns for the masked rows
            df.at[idx, 'st_Max'] = subset_df.values.max()
            df.at[idx, 'st_Min'] = subset_df.values.min()

    return df

In [23]:
def st_trade_type(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['trade_type'] = np.nan

    df['Date'] = pd.to_datetime(df['Date'])
    df['Entry_Date'] = pd.to_datetime(df['Entry_Date'])
    df['st_Exit_Date'] = pd.to_datetime(df['st_Exit_Date'])

    trade_entries = df[df['Type'].isin(['Long', 'Short'])].copy()

    for index, trade in trade_entries.iterrows():
        entry_date = trade['Entry_Date']
        exit_date = trade['st_Exit_Date']
        trade_type = trade['Type']

        # Find the index of the row before the Exit_Date
        exit_rows = df.index[df['Date'] == exit_date]
        if not exit_rows.empty and exit_rows[0] > 0:
            prev_date = df.at[exit_rows[0] - 1, 'Date']
        else:
            # If no valid exit date, consider the trade open until the end of the data
            prev_date = df['Date'].max()

        mask = (df['Date'] >= entry_date) & (df['Date'] <= prev_date)

        if trade_type == 'Long':
            df.loc[mask, 'trade type'] = 1
        elif trade_type == 'Short':
            df.loc[mask, 'trade type'] = 0

    return df

In [24]:
def st_pnl(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df['st_Exit_Date'] = pd.to_datetime(df['st_Exit_Date'])
    df['st_PnL'] = np.nan

    pnl_by_date = df.set_index('Date')['st_row_PnL_close']
    entries = df[df['Type'].isin(['Long', 'Short'])]
    for idx, trade in entries.iterrows():
        exit_dt = trade['st_Exit_Date']
        exit_rows = df.index[df['Date'] == exit_dt]
        if not exit_rows.empty and exit_rows[0] > 0:
            prev_date = df.at[exit_rows[0] - 1, 'Date']
            df.at[idx, 'st_PnL'] = pnl_by_date.get(prev_date, np.nan)

    return df

In [25]:
def PnL_trade_analytics_table(df: pd.DataFrame):
    """
    Calcula métricas de rendimiento de trades con los siguientes formatos:
      • Total PnL, Avg Profit y Avg Loss → $ con separador de miles, 0 decimales
      • Win (%) y Loss (%) → 2 decimales + símbolo %
      • Profit Factor → 2 decimales
      • # Days y conteos → separador de miles
    """
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])

    # ──────────────────── Cálculo numérico ────────────────────
    analytics = pd.DataFrame(index=['Strategy'])
    analytics['# Days'] = (df['Date'].max() - df['Date'].min()).days
    analytics['Total PnL']  = round(df.st_PnL.sum(), 0)
    analytics['Total Trades'] = df.st_PnL.count()
    analytics['Number of Winners'] = (df.st_PnL > 0).sum()
    analytics['Number of Losers']  = (df.st_PnL <= 0).sum()
    analytics['Win (%)']  = round(100 * analytics.at['Strategy','Number of Winners'] /
                                  analytics.at['Strategy','Total Trades'], 2)
    analytics['Loss (%)'] = round(100 * analytics.at['Strategy','Number of Losers']  /
                                  analytics.at['Strategy','Total Trades'], 2)
    analytics['Average Profit of Winning Trade'] = round(df.loc[df.st_PnL > 0, 'st_PnL'].mean(), 0)
    analytics['Average Loss of Losing Trade']    = round(abs(df.loc[df.st_PnL < 0, 'st_PnL'].mean()), 0)

    if {'Entry_Date','st_Exit_Date'}.issubset(df.columns):
        df['Entry_Date']   = pd.to_datetime(df['Entry_Date'])
        df['st_Exit_Date'] = pd.to_datetime(df['st_Exit_Date'])
        analytics['Average Holding Time'] = (df['st_Exit_Date'] - df['Entry_Date']).mean()

    tot_profit   = df.loc[df.st_PnL > 0, 'st_PnL'].sum()
    tot_loss_abs = abs(df.loc[df.st_PnL < 0, 'st_PnL'].sum())
    analytics['Profit Factor'] = round(tot_profit / tot_loss_abs, 2) if tot_loss_abs else np.nan

    # ──────────────────── Formato visual ────────────────────
    def _fmt(val, metric):
        if metric in ('Win (%)', 'Loss (%)'):
            return f'{val:,.2f}%'
        if metric in ('Total PnL',
                      'Average Profit of Winning Trade',
                      'Average Loss of Losing Trade'):
            return f'$ {val:,.0f}'
        if metric in ('# Days', 'Total Trades', 'Number of Winners', 'Number of Losers'):
            return f'{val:,.0f}'
        if metric in ('Profit Factor',):
            return f'{val:,.2f}'
        return val

    styled = analytics.T.copy()
    styled['Strategy'] = [ _fmt(v, idx) for idx, v in styled['Strategy'].items() ]

    return (styled.style
                  .set_caption('')
                  .set_table_styles([
                      {'selector':'th','props':[('font-weight','bold'), ('text-align','center')]},
                      {'selector':'td','props':[('text-align','center')]},
                      {'selector':'thead th', 'props':[('text-align','center')]}
                  ]))


In [26]:
def st_add_atr_scaled_pnls(
    df: pd.DataFrame,
    *,
    atr_col: str = "ATR",
    usd_per_point: float = 100.0,
    atr_len: int = 14
) -> pd.DataFrame:
    """
    - Elimina ÚNICAMENTE la columna 'trade_type' si existe.
    - Crea 'st_atr_PnL' y 'st_atr_max_PnL' en TODAS las filas del bloque de cada trade,
      expresando st_PnL y st_Max en múltiplos de ATR (usando el ATR de la fila de apertura).
    - Si no existe 'ATR', lo calcula (ATR(14) EWM).
    """
    df = df.copy()

    # Asegurar tipos y orden
    for c in ("Date", "Entry_Date", "st_Exit_Date"):
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")
    df = df.sort_values("Date").reset_index(drop=True)

    # Eliminar SOLO 'trade_type'
    if "trade_type" in df.columns:
        df.drop(columns=["trade_type"], inplace=True)

    # Asegurar ATR
    if (atr_col not in df.columns) or (df[atr_col].isna().all()):
        high = pd.to_numeric(df["High"], errors="coerce")
        low  = pd.to_numeric(df["Low"],  errors="coerce")
        close = pd.to_numeric(df["Close"], errors="coerce")
        prev_close = close.shift(1)
        tr = pd.concat([(high - low), (high - prev_close).abs(), (low - prev_close).abs()], axis=1).max(axis=1)
        df[atr_col] = tr.ewm(alpha=1.0/atr_len, adjust=False, min_periods=atr_len).mean()

    # Crear columnas destino
    df['st_atr_PnL'] = np.nan
    df['st_atr_max_PnL'] = np.nan

    # Recorrer entradas
    entries = df[df["Type"].isin(["Long", "Short"])].copy()
    if entries.empty:
        return df

    for _, trade in entries.iterrows():
        entry_dt = trade["Entry_Date"]
        exit_dt  = trade["st_Exit_Date"]

        base_row = df.loc[df["Date"] == entry_dt]
        if base_row.empty or pd.isna(base_row.iloc[0][atr_col]):
            continue
        atr_base = float(base_row.iloc[0][atr_col])
        atr_usd = atr_base * usd_per_point if np.isfinite(atr_base) else np.nan
        if not np.isfinite(atr_usd) or atr_usd == 0:
            continue

        # Barra previa al exit para delimitar bloque
        exit_rows = df.index[df["Date"] == exit_dt]
        if not exit_rows.empty and exit_rows[0] > 0:
            prev_date = df.at[exit_rows[0] - 1, "Date"]
        else:
            prev_date = df["Date"].max()

        mask = (df["Date"] >= entry_dt) & (df["Date"] <= prev_date)

        # st_atr_pnl: usar st_PnL del trade (USD) / (ATR_base * usd_per_point)
        pnl_usd = trade.get("st_PnL", np.nan)
        # reconstrucción si falta st_PnL
        if not np.isfinite(pnl_usd):
            base_close = float(base_row.iloc[0]["Close"]) if "Close" in base_row.columns else np.nan
            row_prev = df.loc[df["Date"] == prev_date]
            if row_prev.empty or not np.isfinite(base_close):
                continue
            close_prev = float(row_prev.iloc[0]["Close"])
            if trade["Type"] == "Long":
                pnl_usd = (close_prev - base_close) * usd_per_point
            else:
                pnl_usd = (base_close - close_prev) * usd_per_point
        atr_pnl = pnl_usd / atr_usd if np.isfinite(pnl_usd) else np.nan

        # st_atr_max_pnl: usar st_Max (USD) / (ATR_base * usd_per_point)
        max_usd = trade.get("st_Max", np.nan)
        if not np.isfinite(max_usd):
            cols = [c for c in ("st_row_PnL_close", "st_row_PnL_low", "st_row_PnL_high") if c in df.columns]
            if cols:
                subset = df.loc[mask, cols]
                if not subset.empty:
                    max_usd = np.nanmax(subset.values)
        atr_max_pnl = (max_usd / atr_usd) if np.isfinite(max_usd) else np.nan

        # Rellenar en todas las filas del bloque
        df.loc[mask, 'st_atr_PnL'] = atr_pnl
        df.loc[mask, 'st_atr_max_PnL'] = atr_max_pnl

    return df

In [27]:
import numpy as np
import pandas as pd

def st_add_atr_multipliers_from_prices(
    df: pd.DataFrame,
    *,
    atr_col: str = "ATR",
    atr_len: int = 14
) -> pd.DataFrame:
    """
    Compute ATR-multiplier columns based on PRICES (not USD PnL), per your formulas:

      LONG:
        price_max_profit  = entry_close + (ATR_entry * atr_multiplier_max)
        price_close_trade = entry_close + (ATR_entry * atr_multiplier_close)

      SHORT:
        price_max_profit  = entry_close - (ATR_entry * atr_multiplier_max)
        price_close_trade = entry_close - (ATR_entry * atr_multiplier_close)

    Definitions used:
      • entry_close  := Close at Entry_Date
      • ATR_entry    := ATR value at Entry_Date
      • price_close_trade := Close at the bar immediately BEFORE st_Exit_Date
      • price_max_profit  := For Long → max High within [Entry_Date, bar_before_exit]
                             For Short → min Low  within [Entry_Date, bar_before_exit]

    The resulting multipliers are written across every row of each trade block:
      • st_atr_PnL      → multiplier to reach the close-trade price
      • st_atr_max_PnL  → multiplier to reach the max-profit price

    If ATR is missing, it is computed as an EWM (Wilder-style) of True Range.
    """

    out = df.copy()

    # Ensure types and order
    for c in ("Date", "Entry_Date", "st_Exit_Date"):
        if c in out.columns:
            out[c] = pd.to_datetime(out[c], errors="coerce")
    out = out.sort_values("Date").reset_index(drop=True)

    # Ensure numeric price columns
    for c in ("Close", "High", "Low"):
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce")

    # Ensure ATR exists (EWM of True Range if missing or all-NaN)
    if (atr_col not in out.columns) or (out[atr_col].isna().all()):
        high = out["High"]
        low  = out["Low"]
        close = out["Close"]
        prev_close = close.shift(1)
        tr = pd.concat([(high - low),
                        (high - prev_close).abs(),
                        (low  - prev_close).abs()], axis=1).max(axis=1)
        out[atr_col] = tr.ewm(alpha=1.0/atr_len, adjust=False, min_periods=atr_len).mean()

    # Prepare destination columns
    out["st_atr_PnL"] = np.nan
    out["st_atr_max_PnL"] = np.nan

    # Iterate over trades (rows where Type is 'Long' or 'Short')
    if "Type" not in out.columns:
        return out

    trades = out[out["Type"].isin(["Long", "Short"])].copy()
    for _, tr in trades.iterrows():
        entry_dt = tr.get("Entry_Date", pd.NaT)
        exit_dt  = tr.get("st_Exit_Date", pd.NaT)
        side     = tr.get("Type", None)

        if pd.isna(entry_dt) or side not in ("Long", "Short"):
            continue

        # Base row at entry
        base_row = out.loc[out["Date"] == entry_dt]
        if base_row.empty:
            continue

        entry_close = float(base_row.iloc[0]["Close"])
        atr_entry   = float(base_row.iloc[0][atr_col]) if atr_col in base_row.columns else np.nan
        if not np.isfinite(entry_close) or not np.isfinite(atr_entry) or atr_entry <= 0:
            continue

        # Determine bar_before_exit to delimit the trade block
        if pd.notna(exit_dt):
            exit_idx = out.index[out["Date"] == exit_dt]
            if len(exit_idx) > 0 and exit_idx[0] > 0:
                bar_before_exit_date = out.at[exit_idx[0] - 1, "Date"]
            else:
                # If exit is the first row or not found, consider up to the last available row
                bar_before_exit_date = out["Date"].max()
        else:
            # If no exit, consider up to the last available row
            bar_before_exit_date = out["Date"].max()

        # Mask for the trade block [Entry_Date, bar_before_exit]
        mask = (out["Date"] >= entry_dt) & (out["Date"] <= bar_before_exit_date)
        block = out.loc[mask, ["High", "Low", "Close"]].dropna()
        if block.empty:
            continue

        # Prices of interest
        price_close_trade = float(out.loc[out["Date"] == bar_before_exit_date, "Close"].iloc[0])

        if side == "Long":
            # Max favorable price in block is the highest High
            price_max_profit = float(block["High"].max())
            atr_mult_close = (price_close_trade - entry_close) / atr_entry
            atr_mult_max   = (price_max_profit  - entry_close) / atr_entry
        else:  # Short
            # Max favorable price for Short is the lowest Low
            price_max_profit = float(block["Low"].min())
            atr_mult_close = (entry_close - price_close_trade) / atr_entry
            atr_mult_max   = (entry_close - price_max_profit ) / atr_entry

        # Write the same multiplier across the trade block
        out.loc[mask, "st_atr_PnL"] = atr_mult_close
        out.loc[mask, "st_atr_max_PnL"] = atr_mult_max

    return out

## Implementation

In [28]:
data = st_exit_date(data)
print(data.shape)
data = st_trade_type(data)
print(data.shape)
data = st_duration(data)
print(data.shape)
data = st_pnl_close(data)
print(data.shape)
data = st_pnl_high(data)
print(data.shape)
data = st_pnl_low(data)
print(data.shape)
data = st_max_min(data)
print(data.shape)
data = st_pnl(data)
print(data.shape)
data = st_add_atr_scaled_pnls(data, atr_col="ATR", usd_per_point=100.0, atr_len=14)
print(data.columns)
data = st_add_atr_multipliers_from_prices(data)
print(data.columns)

(20000, 18)
(20000, 20)
(20000, 21)
(20000, 22)
(20000, 23)
(20000, 25)
(20000, 27)
(20000, 28)
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type',
       'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low',
       'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL',
       'st_atr_max_PnL'],
      dtype='object')
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type',
       'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low',
       'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL',
       'st_atr_max_PnL'],
      dtype='object')


In [29]:
columns_to_see = ['Date', 'Close','ATR','trade type', 'Open_Trade',
                  'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL', 'st_atr_max_PnL']

data.loc[data['Open_Trade'].notna(), columns_to_see]
#data.loc[((data['Open_Trade'].notna()) & (data['Type']=='Short')), columns_to_see].head(10)


,Date,Close,ATR,trade type,Open_Trade,st_Max,st_Min,st_PnL,st_atr_PnL,st_atr_max_PnL
1,2025-04-15 04:25:00,3223.35,2.405472,0.0,-1.0,613.0,-261.0,-38.0,-0.157973,2.548356
8,2025-04-15 05:00:00,3223.86,2.649200,1.0,1.0,34.0,-199.0,0.0,0.000000,0.128341
9,2025-04-15 05:05:00,3221.15,2.707828,0.0,-1.0,236.0,-311.0,-104.0,-0.384072,0.871547
15,2025-04-15 05:35:00,3222.59,2.576563,1.0,1.0,551.0,-259.0,199.0,2.006549,3.073862
28,2025-04-15 06:40:00,3225.77,2.187297,1.0,1.0,474.0,-259.0,199.0,0.909799,2.167058
...,...,...,...,...,...,...,...,...,...,...
19948,2025-07-25 19:40:00,3329.91,2.601626,1.0,1.0,946.0,-218.0,91.0,3.747656,3.993656
19972,2025-07-25 21:40:00,3338.75,1.423972,1.0,1.0,155.0,-68.0,91.0,0.639057,1.088504
19982,2025-07-25 22:30:00,3339.28,1.100161,0.0,-1.0,262.0,-77.0,198.0,1.799737,2.381471
19994,2025-07-25 23:30:00,3338.59,1.138175,1.0,1.0,55.0,-149.0,10.0,0.087860,0.483230


In [30]:
data.to_csv(root_data+'Results/'+symbol+'_'+strategy+'_'+time_frame+'_Original.csv')

In [31]:
data_st_pnl = data.copy()
data_st_pnl = data_st_pnl[data_st_pnl['Entry_Date'].notna()]
#data_sl_pnl.head(5)

In [32]:
trade_analysis = PnL_trade_analytics_table(data_st_pnl)
trade_analysis

,Strategy
# Days,101
Total PnL,"$ 327,297"
Total Trades,"2,953"
Number of Winners,"1,215"
Number of Losers,"1,738"
Win (%),41.14%
Loss (%),58.86%
Average Profit of Winning Trade,$ 459
Average Loss of Losing Trade,$ 166
Average Holding Time,0 days 01:08:34.256688113


In [33]:
print(data_st_pnl['st_PnL'].sum())

327297.00000000047


# Genetic_Strategy_Optimization

In [34]:

# === GA para optimizar length_1..length_4 + smooth_4 ===
import numpy as np
import pandas as pd
from typing import Tuple, List, Dict
from tqdm.auto import tqdm

# ----------------------------
# 1) EVALUACIÓN DEL FITNESS
# ----------------------------
def evaluate_lengths(data: pd.DataFrame,
                     length_1: int,
                     length_2: int,
                     length_3: int,
                     length_4: int,
                     smooth_4: int,
                     smooth_1: int = 3,
                     smooth_2: int = 3,
                     smooth_3: int = 3) -> float:
    """
    Recalcula kalman lines y señales con los parámetros dados y retorna Total PnL (suma de st_PnL).
    Apertura: pendientes de kal_1..kal_3
    Cierre:   pendiente de kal_4 (configurada con smooth_4)
    NO modifica el DataFrame original.
    Requiere en el scope: kalman_line, st_exit_date, st_pnl_close, st_pnl
    """
    # --- base y orden ---
    df = data[['Date', 'High', 'Low', 'Close']].copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)

    # --- kalman lines ---
    df['kal_1'] = kalman_line(df['Close'], length_1, smooth_1)
    df['kal_2'] = kalman_line(df['Close'], length_2, smooth_2)
    df['kal_3'] = kalman_line(df['Close'], length_3, smooth_3)
    df['kal_4'] = kalman_line(df['Close'], length_4, smooth_4)

    # --- señales de apertura: pendiente positiva/negativa en 1..3 ---
    bullish = (df['kal_1'] > df['kal_1'].shift()) & \
              (df['kal_2'] > df['kal_2'].shift()) & \
              (df['kal_3'] > df['kal_3'].shift())

    bearish = (df['kal_1'] < df['kal_1'].shift()) & \
              (df['kal_2'] < df['kal_2'].shift()) & \
              (df['kal_3'] < df['kal_3'].shift())

    open_raw = np.where(bullish, 1, np.where(bearish, -1, np.nan))
    # Evitar repetición consecutiva del mismo valor
    df['Open_Trade'] = np.where(open_raw == pd.Series(open_raw).shift(1), np.nan, open_raw)

    # --- metadata de entrada ---
    df['Entry_Date'] = pd.NaT
    df.loc[df['Open_Trade'] ==  1, 'Entry_Date'] = df['Date']
    df.loc[df['Open_Trade'] == -1, 'Entry_Date'] = df['Date']

    df['Type'] = np.nan
    df.loc[df['Open_Trade'] ==  1, 'Type'] = 'Long'
    df.loc[df['Open_Trade'] == -1, 'Type'] = 'Short'

    # --- señal de cierre: pendiente de kal_4 ---
    #   Close long  = -1 cuando pendiente < 0
    #   Close short =  1 cuando pendiente > 0
    close_raw = np.where(df['kal_4'] < df['kal_4'].shift(), -1,
                 np.where(df['kal_4'] > df['kal_4'].shift(),  1, np.nan))
    df['st_Close_Trade'] = np.where(close_raw == pd.Series(close_raw).shift(1), np.nan, close_raw)

    # --- pipeline de salidas + PnL (tus funciones) ---
    df = st_exit_date(df)
    df = st_pnl_close(df)
    df = st_pnl(df)

    # Fitness = Total PnL de los trades (si no hay, penaliza)
    trades = df[df['Entry_Date'].notna()]
    total_pnl = trades['st_PnL'].sum(min_count=1)
    if pd.isna(total_pnl):
        return float('-inf')
    return float(total_pnl)


# ----------------------------
# 2) GA SOBRE ÍNDICES (5 genes)
# ----------------------------
def build_int_grid(start: int, stop: int, step: int) -> np.ndarray:
    """Grilla inclusiva [start, stop] con paso 'step'."""
    return np.arange(start, stop + 1, step, dtype=int)


class GALengthOptimizer:
    """
    GA para 5 parámetros discretos:
      length_1..length_4 + smooth_4
    Sin restricciones de orden (los rangos de lengths están separados; smooth_4 es independiente).
    """
    def __init__(self,
                 data: pd.DataFrame,
                 len1_vals: np.ndarray,
                 len2_vals: np.ndarray,
                 len3_vals: np.ndarray,
                 len4_vals: np.ndarray,
                 s4_vals:  np.ndarray,
                 pop_size: int = 60,
                 generations: int = 30,
                 tournament_k: int = 3,
                 elite_size: int = 4,
                 mutation_rate: float = 0.40,
                 seed: int = 42,
                 log_csv_path: str = 'ga_lengths_runs.csv'):
        self.data = data
        self.len1_vals = np.array(len1_vals, dtype=int)
        self.len2_vals = np.array(len2_vals, dtype=int)
        self.len3_vals = np.array(len3_vals, dtype=int)
        self.len4_vals = np.array(len4_vals, dtype=int)
        self.s4_vals   = np.array(s4_vals,  dtype=int)

        self.pop_size = pop_size
        self.generations = generations
        self.tournament_k = tournament_k
        self.elite_size = elite_size
        self.mutation_rate = mutation_rate
        self.seed = seed
        self.rng = np.random.default_rng(seed)

        self.n1 = len(self.len1_vals)
        self.n2 = len(self.len2_vals)
        self.n3 = len(self.len3_vals)
        self.n4 = len(self.len4_vals)
        self.ns = len(self.s4_vals)
        if min(self.n1, self.n2, self.n3, self.n4, self.ns) <= 0:
            raise ValueError("Alguna grilla está vacía.")

        # caché y logs
        self.cache: Dict[Tuple[int, int, int, int, int], float] = {}
        self.run_logs: List[dict] = []
        self.run_counter = 0
        self.log_csv_path = log_csv_path

    # ---- utilidades ----
    def idxs_to_params(self, ind: Tuple[int, int, int, int, int]) -> Tuple[int, int, int, int, int]:
        i, j, k, h, s = ind
        return (int(self.len1_vals[i]),
                int(self.len2_vals[j]),
                int(self.len3_vals[k]),
                int(self.len4_vals[h]),
                int(self.s4_vals[s]))

    def _clip(self, i: int, j: int, k: int, h: int, s: int) -> Tuple[int, int, int, int, int]:
        i = max(0, min(self.n1 - 1, int(i)))
        j = max(0, min(self.n2 - 1, int(j)))
        k = max(0, min(self.n3 - 1, int(k)))
        h = max(0, min(self.n4 - 1, int(h)))
        s = max(0, min(self.ns - 1, int(s)))
        return i, j, k, h, s

    def random_individual(self) -> Tuple[int, int, int, int, int]:
        return (int(self.rng.integers(0, self.n1)),
                int(self.rng.integers(0, self.n2)),
                int(self.rng.integers(0, self.n3)),
                int(self.rng.integers(0, self.n4)),
                int(self.rng.integers(0, self.ns)))

    def crossover(self,
                  p1: Tuple[int, int, int, int, int],
                  p2: Tuple[int, int, int, int, int]) -> Tuple[int, int, int, int, int]:
        # punto de cruce 1..4
        cx_point = int(self.rng.integers(1, 5))
        child = tuple(list(p1)[:cx_point] + list(p2)[cx_point:])
        return self._clip(*child)

    def mutate(self, ind: Tuple[int, int, int, int, int]) -> Tuple[int, int, int, int, int]:
        # mutación en índice con deltas cortas
        deltas = (-2, -1, 1, 2)
        i, j, k, h, s = ind
        if self.rng.random() < self.mutation_rate:
            i += int(self.rng.choice(deltas))
        if self.rng.random() < self.mutation_rate:
            j += int(self.rng.choice(deltas))
        if self.rng.random() < self.mutation_rate:
            k += int(self.rng.choice(deltas))
        if self.rng.random() < self.mutation_rate:
            h += int(self.rng.choice(deltas))
        if self.rng.random() < self.mutation_rate:
            s += int(self.rng.choice((-1, 1)))  # smooth_4 tiene paso 1, usamos delta ±1
        return self._clip(i, j, k, h, s)

    def tournament_select(self,
                          population: List[Tuple[int, int, int, int, int]],
                          fitnesses: Dict[Tuple[int, int, int, int, int], float]
                         ) -> Tuple[int, int, int, int, int]:
        idxs = self.rng.choice(len(population), size=self.tournament_k, replace=False)
        contenders = [population[int(ix)] for ix in idxs]
        best = max(contenders, key=lambda ind: fitnesses[ind])
        return best

    def eval_individual(self, ind: Tuple[int, int, int, int, int]) -> float:
        if ind in self.cache:
            return self.cache[ind]
        L1, L2, L3, L4, S4 = self.idxs_to_params(ind)
        fitness = evaluate_lengths(self.data, L1, L2, L3, L4, S4)
        self.cache[ind] = fitness
        self.run_counter += 1
        self.run_logs.append({
            'run': self.run_counter,
            'length_1': L1,
            'length_2': L2,
            'length_3': L3,
            'length_4': L4,
            'smooth_4': S4,
            'total_pnl': fitness
        })
        return fitness

    def optimize(self) -> Tuple[Tuple[int, int, int, int, int], float, pd.DataFrame]:
        # población inicial (individuos únicos)
        population: List[Tuple[int, int, int, int, int]] = []
        seen = set()
        while len(population) < self.pop_size:
            ind = self.random_individual()
            if ind not in seen:
                seen.add(ind)
                population.append(ind)

        # evaluación inicial
        fitnesses: Dict[Tuple[int, int, int, int, int], float] = {}
        for ind in tqdm(population, desc="Evaluación inicial", unit="ind", leave=False, mininterval=0.1):
            fitnesses[ind] = self.eval_individual(ind)

        best_ind = max(population, key=lambda x: fitnesses[x])
        best_fit = fitnesses[best_ind]

        # generaciones
        for _ in tqdm(range(self.generations), desc="Generaciones", unit="gen", mininterval=0.1):
            elites = sorted(population, key=lambda x: fitnesses[x], reverse=True)[:self.elite_size]

            offspring: List[Tuple[int, int, int, int, int]] = []
            while len(offspring) + len(elites) < self.pop_size:
                p1 = self.tournament_select(population, fitnesses)
                p2 = self.tournament_select(population, fitnesses)
                child = self.crossover(p1, p2)
                child = self.mutate(child)
                offspring.append(child)

            population = elites + offspring

            new_inds = [ind for ind in population if ind not in fitnesses]
            for ind in tqdm(new_inds, desc=" Eval nuevos", unit="ind", leave=False, mininterval=0.1):
                fitnesses[ind] = self.eval_individual(ind)

            gen_best_ind = max(population, key=lambda x: fitnesses[x])
            gen_best_fit = fitnesses[gen_best_ind]
            if gen_best_fit > best_fit:
                best_fit = gen_best_fit
                best_ind = gen_best_ind

        best_params = self.idxs_to_params(best_ind)
        log_df = pd.DataFrame(self.run_logs,
                              columns=['run','length_1','length_2','length_3','length_4','smooth_4','total_pnl'])
        log_df.to_csv(self.log_csv_path, index=False)
        return best_params, float(best_fit), log_df


# ----------------------------
# 3) FUNCIÓN DE CONVENIENCIA
# ----------------------------
def ga_optimize_lengths(data: pd.DataFrame,
                        pop_size: int = 60,
                        generations: int = 30,
                        tournament_k: int = 3,
                        elite_size: int = 4,
                        mutation_rate: float = 0.40,
                        seed: int = 42,
                        log_csv_path: str = 'ga_lengths_runs.csv'):
    """
    Lanza el GA con las rejillas pedidas:
      length_1: 100..300 paso 10
      length_2: 400..600 paso 10
      length_3: 700..900 paso 10
      length_4: 700..1200 paso 10
      smooth_4 : 2..7 paso 1
    """
    len1_vals = build_int_grid(100, 300, 10)
    len2_vals = build_int_grid(400, 600, 10)
    len3_vals = build_int_grid(700, 900, 10)
    len4_vals = build_int_grid(700, 1200, 10)
    s4_vals   = build_int_grid(2, 7, 1)

    ga = GALengthOptimizer(
        data=data,
        len1_vals=len1_vals,
        len2_vals=len2_vals,
        len3_vals=len3_vals,
        len4_vals=len4_vals,
        s4_vals=s4_vals,
        pop_size=pop_size,
        generations=generations,
        tournament_k=tournament_k,
        elite_size=elite_size,
        mutation_rate=mutation_rate,
        seed=seed,
        log_csv_path=log_csv_path
    )
    return ga.optimize()


In [35]:
# 1) Ejecuta el GA
(best_L1, best_L2, best_L3, best_L4, best_S4), best_pnl, ga_log = ga_optimize_lengths(
    data, pop_size=60, generations=30, seed=42, log_csv_path='ga_lengths_runs.csv')
print("Mejores params:",
      best_L1, best_L2, best_L3, best_L4, "smooth_4=", best_S4, "| Total PnL:", best_pnl)

Evaluación inicial:   0%|          | 0/60 [00:00<?, ?ind/s]

Generaciones:   0%|          | 0/30 [00:00<?, ?gen/s]

 Eval nuevos:   0%|          | 0/55 [00:00<?, ?ind/s]

 Eval nuevos:   0%|          | 0/55 [00:00<?, ?ind/s]

 Eval nuevos:   0%|          | 0/56 [00:00<?, ?ind/s]

...

 Eval nuevos:   0%|          | 0/40 [00:00<?, ?ind/s]

 Eval nuevos:   0%|          | 0/37 [00:00<?, ?ind/s]

 Eval nuevos:   0%|          | 0/35 [00:00<?, ?ind/s]

 Eval nuevos:   0%|          | 0/35 [00:00<?, ?ind/s]

Mejores params: 300 520 710 1130 smooth_4= 7 | Total PnL: 393948.0000000001


# PnL_Strategy_Genetic

In [79]:

length_1 = 300
length_2 = 520
length_3 = 710
length_4 = 1130

smooth_1 = 3
smooth_2 = 3
smooth_3 = 3
smooth_4 = 7

data['kal_1'] = kalman_line(data['Close'],length_1, smooth_1)
data['kal_2'] = kalman_line(data['Close'],length_2, smooth_2)
data['kal_3'] = kalman_line(data['Close'],length_3, smooth_3)
data['kal_4'] = kalman_line(data['Close'],length_4, smooth_4)

In [80]:
bullish_condition = (data['kal_1'] > data['kal_1'].shift()) & \
                    (data['kal_2'] > data['kal_2'].shift()) & \
                    (data['kal_3'] > data['kal_3'].shift())

bearish_condition = (data['kal_1'] < data['kal_1'].shift()) & \
                    (data['kal_2'] < data['kal_2'].shift()) & \
                    (data['kal_3'] < data['kal_3'].shift())

data['Open_Trade_1'] = np.where(bullish_condition, 1, np.where(bearish_condition, -1, np.nan))
data['Open_Trade'] = np.where(data['Open_Trade_1'] == data['Open_Trade_1'].shift(1), np.nan, data['Open_Trade_1'])

close_bearish_condition = data['Open_Trade'] == 1
close_bullish_condition = data['Open_Trade'] == -1

data['Close_Trade_1'] = np.where(data['kal_4'] < data['kal_4'].shift(), -1, np.where(data['kal_4'] > data['kal_4'].shift(), 1, np.nan))
data['Close_Trade'] = np.where(data['Close_Trade_1'] == data['Close_Trade_1'].shift(1), np.nan, data['Close_Trade_1'])

data = data.drop(columns=['Open_Trade_1', 'Close_Trade_1'])
#data

In [81]:
data['Entry_Date'] = pd.NaT
data.loc[data['Open_Trade'] ==  1, 'Entry_Date'] = data['Date'][data['Open_Trade'] == 1]
data.loc[data['Open_Trade'] == -1, 'Entry_Date'] = data['Date'][data['Open_Trade'] == -1]

data['Type'] = np.nan
data.loc[data['Open_Trade'] ==  1, 'Type'] = 'Long'
data.loc[data['Open_Trade'] == -1, 'Type'] = 'Short'
#data.loc[data['Open_Trade'] == -1, 'Type'] = 'Short'
#data['Type'] = data['Type'].ffill()

data['Trade_Number'] = np.nan
start_mask = data["Entry_Date"].notna() & data["Type"].notna()

data["Trade_Number"] = start_mask.cumsum()
first_trade = data["Trade_Number"].ne(0).idxmax()
data.loc[: first_trade - 1, "Trade_Number"] = np.nan
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type',
       'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low',
       'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL',
       'st_atr_max_PnL', 'atr_mult_close', 'atr_mult_high', 'atr_mult_low',
       'atr_dyn', 'atr_PnL', 'atr_Exit_Date', 'atr_Duration', 'atr_PnL_dollar',
       'atr_H_dyn', 'atr_H_PnL', 'atr_H_Exit_Date', 'atr_H_Duration',
       'atr_H_PnL_dollar'],
      dtype='object')

In [82]:
data = st_exit_date(data)
print(data.shape)
data = st_trade_type(data)
print(data.shape)
data = st_duration(data)
print(data.shape)
data = st_pnl_close(data)
print(data.shape)
data = st_pnl_high(data)
print(data.shape)
data = st_pnl_low(data)
print(data.shape)
data = st_max_min(data)
print(data.shape)
data = st_pnl(data)
print(data.shape)
data = st_add_atr_scaled_pnls(data, atr_col="ATR", usd_per_point=100.0, atr_len=14)
print(data.columns)
data = st_add_atr_multipliers_from_prices(data)
print(data.columns)

(20000, 42)
(20000, 43)
(20000, 43)
(20000, 43)
(20000, 43)
(20000, 43)
(20000, 43)
(20000, 43)
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type',
       'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low',
       'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL',
       'st_atr_max_PnL', 'atr_mult_close', 'atr_mult_high', 'atr_mult_low',
       'atr_dyn', 'atr_PnL', 'atr_Exit_Date', 'atr_Duration', 'atr_PnL_dollar',
       'atr_H_dyn', 'atr_H_PnL', 'atr_H_Exit_Date', 'atr_H_Duration',
       'atr_H_PnL_dollar'],
      dtype='object')
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type',
       'st_Duration', 'st_row_PnL_clos

In [83]:
data_st_pnl = data.copy()
data_st_pnl = data_st_pnl[data_st_pnl['Entry_Date'].notna()]
#data_sl_pnl.head(5)

In [84]:
trade_analysis = PnL_trade_analytics_table(data_st_pnl)
trade_analysis

,Strategy
# Days,101
Total PnL,"$ 393,948"
Total Trades,"2,807"
Number of Winners,"1,226"
Number of Losers,"1,581"
Win (%),43.68%
Loss (%),56.32%
Average Profit of Winning Trade,$ 484
Average Loss of Losing Trade,$ 162
Average Holding Time,0 days 01:06:46.768792304


In [85]:
print(data_st_pnl['st_PnL'].sum())

393948.0000000001


In [86]:
data.to_csv(root_data+'Results/'+symbol+'_'+strategy+'_'+time_frame+'_Strategy_Gen_Labels.csv')

In [87]:
data.loc[data['Open_Trade'].notna(),['Date', 'Close','ATR', 'Open_Trade', 'trade type', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL', 'st_atr_max_PnL']]

,Date,Close,ATR,Open_Trade,trade type,st_Max,st_Min,st_PnL,st_atr_PnL,st_atr_max_PnL
1,2025-04-15 04:25:00,3223.35,2.4055,-1.0,0.0,613.0,-261.0,-38.0,-0.157971,2.548327
8,2025-04-15 05:00:00,3223.86,2.6492,1.0,1.0,34.0,-199.0,0.0,0.000000,0.128341
9,2025-04-15 05:05:00,3221.15,2.7078,-1.0,0.0,236.0,-311.0,-104.0,-0.384076,0.871556
15,2025-04-15 05:35:00,3222.59,2.5766,1.0,1.0,551.0,-197.0,261.0,2.006520,3.073818
26,2025-04-15 06:30:00,3225.15,2.2935,1.0,1.0,536.0,-197.0,261.0,1.137999,2.337039
...,...,...,...,...,...,...,...,...,...,...
19948,2025-07-25 19:40:00,3329.91,2.6016,1.0,1.0,946.0,-218.0,91.0,3.747694,3.993696
19972,2025-07-25 21:40:00,3338.75,1.4240,1.0,1.0,155.0,-68.0,91.0,0.639045,1.088483
19982,2025-07-25 22:30:00,3339.28,1.1002,-1.0,0.0,262.0,-77.0,198.0,1.799673,2.381385
19994,2025-07-25 23:30:00,3338.59,1.1382,1.0,1.0,55.0,-149.0,10.0,0.087858,0.483219


# SL_Dyn

## Functions

In [71]:
import numpy as np
import pandas as pd

def SL_PnL(df: pd.DataFrame) -> pd.DataFrame:
    """
    Para cada fila con Type in ['Long','Short'], calcula SL_PnL como:
      - El último valor no-NaN de 'dyn_stoploss' ANTES del primer NaN
        dentro de [Entry_Date, st_Exit_Date].
      - Si no hay NaN en la ventana, usa el último valor no-NaN de la ventana.
      - Si todo es NaN o la ventana está vacía, deja NaN.
    """
    out = df.copy()
    out['SL_PnL'] = np.nan

    entries = out[out['Type'].isin(['Long', 'Short'])]

    for idx, row in entries.iterrows():
        entry = row.get('Entry_Date')
        exit_ = row.get('st_Exit_Date')
        if pd.isna(entry) or pd.isna(exit_):
            continue

        m = (out['Date'] >= entry) & (out['Date'] <= exit_)
        s = out.loc[m, 'dyn_stoploss']

        if s.empty:
            continue

        is_na = s.isna().to_numpy()
        if is_na.any():
            first_na_pos = np.argmax(is_na)  # índice del primer NaN
            before_na = s.iloc[:first_na_pos].dropna()
            sl_val = before_na.iloc[-1] if not before_na.empty else np.nan
        else:
            non_na = s.dropna()
            sl_val = non_na.iloc[-1] if not non_na.empty else np.nan

        out.at[idx, 'SL_PnL'] = sl_val

    return out


def SL_exit_date(df: pd.DataFrame) -> pd.DataFrame:
    """
    Asigna SL_Exit_Date y SL_Duration tomando la fecha de la fila donde se determinó SL_PnL.
    Requiere que la columna 'SL_PnL' ya exista (producida por SL_PnL()).
    """
    if 'SL_PnL' not in df.columns:
        raise ValueError("Debe ejecutar primero SL_PnL() para crear la columna 'SL_PnL'.")

    out = df.copy()
    out['Date'] = pd.to_datetime(out['Date'])
    out['Entry_Date'] = pd.to_datetime(out['Entry_Date'])
    out['st_Exit_Date'] = pd.to_datetime(out['st_Exit_Date'])

    out['SL_Exit_Date'] = pd.NaT
    out['SL_Duration'] = np.nan

    trade_mask = out['Type'].isin(['Long', 'Short'])

    for idx, trade in out.loc[trade_mask].iterrows():
        entry = trade['Entry_Date']
        final_exit = trade['st_Exit_Date']
        sl_value = trade['SL_PnL']

        if pd.isna(entry) or pd.isna(final_exit) or pd.isna(sl_value):
            continue

        window_mask = (out['Date'] >= entry) & (out['Date'] <= final_exit)
        window = out.loc[window_mask]

        if window.empty:
            continue

        # Filas donde el dyn_stoploss coincide con el SL_PnL asignado a la operación
        candidates = window[window['dyn_stoploss'] == sl_value]
        if candidates.empty:
            continue

        # Segunda aparición si existe; si no, la primera
        target = candidates.iloc[1] if len(candidates) > 1 else candidates.iloc[0]
        sl_exit_date = target['Date']

        out.at[idx, 'SL_Exit_Date'] = sl_exit_date
        out.at[idx, 'SL_Duration'] = (sl_exit_date - entry).total_seconds() / 60.0

    return out

# ─────────────────────────────────────────────────────────────
# 3. SL_Duration  ─ minutos entre Entry_Date y SL_Exit_Date
# ─────────────────────────────────────────────────────────────
def SL_Duration(df: pd.DataFrame) -> pd.DataFrame:
    if 'SL_Exit_Date' not in df.columns:
        raise ValueError("La columna 'SL_Exit_Date' no existe. Ejecuta SL_Exit_Date() antes.")

    out = df.copy()
    out['SL_Duration'] = np.nan

    mask = out['Type'].isin(['Long', 'Short'])
    durations = (out.loc[mask, 'SL_Exit_Date'] - out.loc[mask, 'Entry_Date']) \
                  .dt.total_seconds() / 60.0
    out.loc[mask, 'SL_Duration'] = durations

    return out

In [72]:
def SL_trade_analytics_table(df: pd.DataFrame):
    """
    Analiza resultados de trades SL con formatos visuales:
      • Total PnL, Avg Profit y Avg Loss → $ con separador de miles, 0 decimales
      • Win (%) y Loss (%) → 2 decimales + símbolo %
      • Profit Factor → 2 decimales
      • # Days y conteos → separador de miles
      • Estilo visual centrado y limpio
    """
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])

    # ──────────────────── Cálculo numérico ────────────────────
    analytics = pd.DataFrame(index=['Dynamic_SL'])
    analytics['# Days'] = (df['Date'].max() - df['Date'].min()).days
    analytics['Total PnL'] = round(df.SL_PnL.sum(), 0)
    analytics['Total Trades'] = df.SL_PnL.count()
    analytics['Number of Winners'] = (df.SL_PnL > 0).sum()
    analytics['Number of Losers']  = (df.SL_PnL <= 0).sum()
    analytics['Win (%)']  = round(100 * analytics.at['Dynamic_SL', 'Number of Winners'] /
                                  analytics.at['Dynamic_SL', 'Total Trades'], 2)
    analytics['Loss (%)'] = round(100 * analytics.at['Dynamic_SL', 'Number of Losers']  /
                                  analytics.at['Dynamic_SL', 'Total Trades'], 2)
    analytics['Average Profit of Winning Trade'] = round(df.loc[df.SL_PnL > 0, 'SL_PnL'].mean(), 0)
    analytics['Average Loss of Losing Trade']    = round(abs(df.loc[df.SL_PnL < 0, 'SL_PnL'].mean()), 0)

    if {'Entry_Date','SL_Exit_Date'}.issubset(df.columns):
        df['Entry_Date']   = pd.to_datetime(df['Entry_Date'])
        df['SL_Exit_Date'] = pd.to_datetime(df['SL_Exit_Date'])
        analytics['Average Holding Time'] = (df['SL_Exit_Date'] - df['Entry_Date']).mean()

    tot_profit   = df.loc[df.SL_PnL > 0, 'SL_PnL'].sum()
    tot_loss_abs = abs(df.loc[df.SL_PnL < 0, 'SL_PnL'].sum())
    analytics['Profit Factor'] = round(tot_profit / tot_loss_abs, 2) if tot_loss_abs else np.nan

    # ──────────────────── Formato visual ────────────────────
    def _fmt(val, metric):
        if metric in ('Win (%)', 'Loss (%)'):
            return f'{val:,.2f}%'
        if metric in ('Total PnL',
                      'Average Profit of Winning Trade',
                      'Average Loss of Losing Trade'):
            return f'${val:,.0f}'
        if metric in ('# Days', 'Total Trades', 'Number of Winners', 'Number of Losers'):
            return f'{val:,.0f}'
        if metric in ('Profit Factor',):
            return f'{val:,.2f}'
        return val

    styled = analytics.T.copy()
    styled['Dynamic_SL'] = [ _fmt(v, idx) for idx, v in styled['Dynamic_SL'].items() ]

    return (styled.style
                  .set_caption('')
                  .set_table_styles([
                      {'selector': 'th',  'props': [('font-weight','bold'), ('text-align','center')]},
                      {'selector': 'td',  'props': [('text-align','center')]},
                      {'selector': 'thead th', 'props': [('text-align','center')]}
                  ]))

In [73]:
def SL_gen_fit_function(data: pd.DataFrame,
                       gen_initial_sl: float,
                       gen_first_step: float,
                       gen_gap_step: float) -> pd.DataFrame:

    df = SL_dyn(data.copy(), initial_sl=gen_initial_sl,
                   first_step=gen_first_step,
                   gap_first_step=gen_gap_step)

    df = SL_PnL(df)
    df = SL_exit_date(df)
    df = SL_Duration(df)
    total_sl_pnl = df['SL_PnL'].sum()

    summary = pd.DataFrame({
        'gen_initial_sl': [gen_initial_sl],
        'gen_first_step': [gen_first_step],
        'gen_gap_step': [gen_gap_step],
        'total_SL_PnL': [total_sl_pnl]
    })

    return summary

In [74]:

def SL_dyn(data: pd.DataFrame, initial_sl: float = -200, first_step: float = 100,
           gap_first_step: float = 100) -> pd.DataFrame:

    df = data.copy()
    col_name = 'dyn_stoploss'
    df[col_name] = np.nan

    in_trade       = False
    trade_active   = False
    broken         = False
    sl_val         = initial_sl
    next_threshold = first_step

    for idx, row in df.iterrows():

        # ── 1. Apertura de la operación ────────────────────────────
        if pd.notna(row['Entry_Date']):
            in_trade       = True
            trade_active   = True
            broken         = False
            sl_val         = initial_sl
            next_threshold = first_step
            prev_sl        = sl_val

            # Asignamos el SL inicial en la primera fila del trade
            df.at[idx, col_name] = sl_val
            continue  # saltamos el resto de la lógica en esta fila

        if in_trade:
            best_pnl = np.nanmax([
                row['st_row_PnL_high'],
                row['st_row_PnL_low'],
                row['st_row_PnL_close']
            ])
            best_pnl = 0 if np.isnan(best_pnl) else best_pnl

            if trade_active:
                # ── 2. Subir SL cada vez que cruza el umbral ─────────────
                while best_pnl >= next_threshold:
                    sl_val         += gap_first_step
                    next_threshold += gap_first_step

                # ── 3. Si cualquiera de los PnL intrabarra cae bajo prev_sl ──
                if (row['st_row_PnL_high']  < prev_sl or
                    row['st_row_PnL_low']   < prev_sl or
                    row['st_row_PnL_close'] < prev_sl):
                    broken       = True
                    trade_active = False

            # ── 4. Escribir NaN tras la ruptura, sino el sl_val ─────────
            df.at[idx, col_name] = np.nan if broken else sl_val
            prev_sl = sl_val

            # ── 5. Cierre oficial al llegar a st_Exit_Date ────────────
            if pd.notna(row['st_Exit_Date']) and row['Date'] == row['st_Exit_Date']:
                in_trade     = False
                trade_active = False

    return df

## Test

In [75]:
initial_sl = -200
first_step = 100
gap_first_step = 50

data = SL_dyn(data, initial_sl, first_step, gap_first_step)
data = SL_PnL(data)
data = SL_exit_date(data)
data = SL_Duration(data)

In [76]:
data.columns
#data.tail()

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type',
       'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low',
       'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL',
       'st_atr_max_PnL', 'dyn_stoploss', 'SL_PnL', 'SL_Exit_Date',
       'SL_Duration'],
      dtype='object')

In [ ]:
sl_columns = ['Date', 'Type', 'Close', 'SL_Exit_Date', 'SL_Duration','st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_low',
               'dyn_stoploss', 'SL_PnL']

#time_a, time_b = '2025-05-16 12:10:00', '2025-05-16 13:10:00'
time_a, time_b = '2025-05-15 14:50:00', '2025-05-15 18:00:00'

data.loc[data['Open_Trade'].notna(), sl_columns].head(20)
data.loc[(data['Date']>= time_a) & (data['Date']<= time_b), sl_columns]

In [78]:
data_sl_pnl = data.copy()
data_sl_pnl = data_sl_pnl[data_sl_pnl['Entry_Date'].notna()]
#data_sl_pnl.head(5)

In [79]:
trade_analysis = SL_trade_analytics_table(data_sl_pnl)
trade_analysis

,Dynamic_SL
# Days,101
Total PnL,"$-16,050"
Total Trades,"2,805"
Number of Winners,859
Number of Losers,"1,946"
Win (%),30.62%
Loss (%),69.38%
Average Profit of Winning Trade,$359
Average Loss of Losing Trade,$179
Average Holding Time,0 days 00:25:13.903743315


In [80]:
print(data_sl_pnl['SL_PnL'].sum())

-16050.0


## Genetic

In [ ]:

try:
    from tqdm.auto import trange
    _HAS_TQDM = True
except Exception:
    _HAS_TQDM = False

# ================== FUNCTION_1 (Referencia) ==================
def SL_gen_fit_function(data: pd.DataFrame,
                        gen_initial_sl: float,
                        gen_first_step: float,
                        gen_gap_step: float) -> pd.DataFrame:

    df = SL_dyn(data.copy(),
                initial_sl=gen_initial_sl,
                first_step=gen_first_step,
                gap_first_step=gen_gap_step)

    df = SL_PnL(df)
    df = SL_exit_date(df)
    df = SL_Duration(df)
    total_sl_pnl = df['SL_PnL'].sum()

    summary = pd.DataFrame({
        'gen_initial_sl': [gen_initial_sl],
        'gen_first_step': [gen_first_step],
        'gen_gap_step':   [gen_gap_step],
        'total_SL_PnL':   [total_sl_pnl]
    })

    return summary

# ================== CONFIG ESPACIOS DE BÚSQUEDA ==================

GEN_INITIAL_SL = (-500, -100, 10)
GEN_FIRST_STEP = (10, 300, 10)
GEN_GAP_STEP   = (100, 200, 10)

def _discrete_space(low: float, high: float, step: float) -> np.ndarray:
    # Evita errores de punto flotante
    n = int(round((high - low) / step)) + 1
    return np.round(low + np.arange(n) * step, 10)

_SPACE_INIT  = _discrete_space(*GEN_INITIAL_SL)
_SPACE_FIRST = _discrete_space(*GEN_FIRST_STEP)
_SPACE_GAP   = _discrete_space(*GEN_GAP_STEP)


def _random_gene() -> Tuple[float, float, float]:
    return (np.random.choice(_SPACE_INIT),
            np.random.choice(_SPACE_FIRST),
            np.random.choice(_SPACE_GAP))

def _mutate_gene(gene: Tuple[float, float, float],
                 p_mut: float,
                 mut_span: int = 1) -> Tuple[float, float, float]:
    gi, gf, gg = gene
    if np.random.rand() < p_mut:
        gi = _SPACE_INIT[np.clip(np.where(_SPACE_INIT == gi)[0][0] +
                                 np.random.randint(-mut_span, mut_span+1),
                                 0, len(_SPACE_INIT)-1)]
    if np.random.rand() < p_mut:
        gf = _SPACE_FIRST[np.clip(np.where(_SPACE_FIRST == gf)[0][0] +
                                  np.random.randint(-mut_span, mut_span+1),
                                  0, len(_SPACE_FIRST)-1)]
    if np.random.rand() < p_mut:
        gg = _SPACE_GAP[np.clip(np.where(_SPACE_GAP == gg)[0][0] +
                                np.random.randint(-mut_span, mut_span+1),
                                0, len(_SPACE_GAP)-1)]
    return (gi, gf, gg)

def _crossover_gene(p1: Tuple[float, float, float],
                    p2: Tuple[float, float, float]) -> Tuple[float, float, float]:
    mask = np.random.randint(0, 2, 3).astype(bool)
    return tuple(np.where(mask, p1, p2))

# ================== GA PRINCIPAL (con tqdm) ==================
def run_ga_SL(
    data: pd.DataFrame,
    pop_size: int = 30,
    generations: int = 40,
    elitism: int = 2,
    p_crossover: float = 0.8,
    p_mutation: float = 0.2,
    tournament_k: int = 3,
    random_state: int | None = None,
    show_progress: bool = True,
    progress_every: int = 1,
    callback: Callable[[int, Tuple[float,float,float], float, List[Dict[str,Any]]], None] | None = None
) -> Tuple[Dict[str, float], pd.DataFrame]:
    """
    GA para maximizar total_profit. Muestra avance con tqdm o prints.

    Parameters
    ----------
    show_progress : bool
        Si True, imprime/progresa barras.
    progress_every : int
        Cada cuántas generaciones reportar (si no hay tqdm).
    callback : callable
        callback(gen, best_gene, best_fit, results_records)

    Returns
    -------
    best_params : dict
    results_df : DataFrame
    """

    if random_state is not None:
        np.random.seed(random_state)

    # Cache fitness
    fitness_cache: Dict[Tuple[float, float, float], float] = {}

    def evaluate(gene: Tuple[float, float, float]) -> float:
        if gene in fitness_cache:
            return fitness_cache[gene]
        res = SL_gen_fit_function(data, *gene)
        fit = float(res['total_SL_PnL'].iloc[0])
        fitness_cache[gene] = fit
        return fit

    # población inicial
    population: List[Tuple[float, float, float]] = [_random_gene() for _ in range(pop_size)]

    results_records: List[Dict[str, Any]] = []

    def tournament_select() -> Tuple[float, float, float]:
        idxs = np.random.choice(len(population), tournament_k, replace=False)
        contenders = [population[i] for i in idxs]
        fits = [evaluate(c) for c in contenders]
        return contenders[int(np.argmax(fits))]

    iterator = trange(generations, desc="GA SL", leave=False) if (_HAS_TQDM and show_progress) else range(generations)

    for gen in iterator:
        # evaluar y guardar
        for ind in population:
            fit = evaluate(ind)
            results_records.append({
                'generation':      gen,
                'gen_initial_sl': ind[0],
                'gen_first_step':   ind[1],
                'gen_gap_step':     ind[2],
                'total_SL_PnL':    fit
            })

        # ordenar y elitismo
        ranked = sorted(population, key=evaluate, reverse=True)
        best_gene = ranked[0]
        best_fit  = evaluate(best_gene)

        # progreso
        if _HAS_TQDM and show_progress:
            iterator.set_postfix(best_fit=f"{best_fit:,.2f}")
        elif show_progress and (gen % progress_every == 0):
            print(f"[Gen {gen:03d}] best_fit={best_fit:,.2f}  gene={best_gene}", flush=True)

        if callback is not None:
            callback(gen, best_gene, best_fit, results_records)

        # reproducir
        next_pop = ranked[:elitism]
        while len(next_pop) < pop_size:
            parent1 = tournament_select()
            parent2 = tournament_select()
            child = parent1
            if np.random.rand() < p_crossover:
                child = _crossover_gene(parent1, parent2)
            child = _mutate_gene(child, p_mutation)
            next_pop.append(child)

        population = next_pop

    # final
    best = max(population, key=evaluate)
    best_fit = evaluate(best)
    best_params = {
        'gen_initial_sl': best[0],
        'gen_first_step': best[1],
        'gen_gap_step'  : best[2],
        'total_SL_PnL'  : best_fit
    }

    results_df = (pd.DataFrame(results_records)
                    .drop_duplicates(subset=['gen_initial_sl','gen_first_step','gen_gap_step','generation'],
                                     keep='last')
                    .reset_index(drop=True))

    return best_params, results_df

In [ ]:
# ================== EJEMPLO DE USO ==================
best_params, results_SL = run_ga_SL(data, pop_size=40, generations=50, random_state=42)
print(best_params)
results_SL.to_csv(root_data + 'Results/'+symbol+'SL_Genetic_Results_M5.csv')
results_SL.tail()

In [ ]:
print(best_params)

In [ ]:
initial_sl = best_params['gen_initial_sl']
first_step_sl = best_params['gen_first_step']
gap_first_step_sl = best_params['gen_gap_step']

data = SL_dyn(data, initial_sl, first_step_sl, gap_first_step_sl)
data = SL_PnL(data)
data = SL_exit_date(data)
data = SL_Duration(data)
data[f'SL_PnL_{initial_sl}_{first_step}_{gap_first_step}'] = data['SL_PnL'].copy()

In [ ]:
data.to_csv(root_data+'Results/'+symbol+'_M5_Genetic_SL.csv')
data.columns

In [ ]:
sl_columns = ['Date', 'Type', 'Entry_Date', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_low',
               'dyn_stoploss', 'SL_PnL', 'SL_Exit_Date', 'SL_Duration']

#data.loc[data['Entry_Date']!=0, sl_columns]

#data.loc[(data['Open_Trade']!=0) & (data['st_Max']>400), sl_columns]

data.loc[(data['Date']>= '2019-07-08 03:10:00') & (data['Date']<= '2019-07-08 04:30:00'),  sl_columns]
#data.loc[(data['Date']>= '2019-07-08 02:30:00') & (data['Date']<= '2019-07-08 04:30:00'), sl_columns]

#data.loc[(data['Date']>= '2019-08-26 05:05:00') & (data['Date']<= '2019-08-26 08:05:00'),sl_columns]

In [ ]:
data_sl_pnl = data.copy()
data_sl_pnl = data_sl_pnl[data_sl_pnl['Entry_Date'].notna()]
#data_sl_pnl.head(5)

In [ ]:
trade_analysis = SL_trade_analytics_table(data_sl_pnl)
trade_analysis

# ATR

## Functions

In [54]:
def atr_multiples(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula tres métricas expresadas en múltiplos de ATR para cada barra
    comprendida entre la apertura (Entry_Date) y la penúltima barra antes del
    cierre (st_Exit_Date) de cada trade Long/Short.

    Se añaden/actualizan tres columnas:
        • atr_mult_close : usa el precio low (misma lógica del original “close”)
        • atr_mult_high  : usa high
        • atr_mult_low   : usa low

    El DataFrame se modifica in-place y se devuelve por conveniencia.
    """
    # ── asegurar tipos datetime --------------------------------------------
    for col in ('Date', 'Entry_Date', 'st_Exit_Date'):
        df[col] = pd.to_datetime(df[col])

    # ── inicializar columnas destino ---------------------------------------
    for col in ('atr_mult_close', 'atr_mult_high', 'atr_mult_low'):
        if col not in df.columns:
            df[col] = np.nan
        else:
            df[col].values[:] = np.nan   # limpiar si ya existían

    # ── iterar sobre cada trade abierto ------------------------------------
    trades = df[df['Type'].isin(['Long', 'Short'])]
    for _, trade in trades.iterrows():
        entry_dt = trade['Entry_Date']
        exit_dt  = trade['st_Exit_Date']
        base_px  = trade['Close']
        atr_base = trade['ATR']
        is_long  = trade['Type'] == 'Long'

        # fila de salida y fecha previa
        exit_idx = df.index[df['Date'] == exit_dt]
        if exit_idx.empty or exit_idx[0] == 0:
            continue                    # no hay barra previa para medir
        prev_date = df.at[exit_idx[0] - 1, 'Date']

        # máscara de barras a rellenar
        mask = (df['Date'] >= entry_dt) & (df['Date'] <= prev_date)

        # --- close (usa low como en función original) ----------------------
        if is_long:
            df.loc[mask, 'atr_mult_close'] = (df.loc[mask, 'Low']  - base_px) / atr_base
            df.loc[mask, 'atr_mult_high']  = (df.loc[mask, 'High'] - base_px) / atr_base
            df.loc[mask, 'atr_mult_low']   = (df.loc[mask, 'Low']  - base_px) / atr_base
        else:  # Short
            df.loc[mask, 'atr_mult_close'] = (base_px - df.loc[mask, 'Low'])  / atr_base
            df.loc[mask, 'atr_mult_high']  = (base_px - df.loc[mask, 'High']) / atr_base
            df.loc[mask, 'atr_mult_low']   = (base_px - df.loc[mask, 'Low'])  / atr_base

    return df

In [55]:
def atr_PnL(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['atr_PnL'] = np.nan

    entries = out[out['Type'].isin(['Long', 'Short'])]

    for idx, row in entries.iterrows():
        start = row['Entry_Date']
        end   = row['st_Exit_Date']
        if pd.isna(end):
            continue

        m = (out['Date'] >= start) & (out['Date'] <= end)
        if not m.any():
            continue

        # Serie numérica de la ventana del trade
        trade_window = pd.to_numeric(out.loc[m, 'atr_dyn'], errors='coerce')

        vals = trade_window.to_numpy()
        isn = np.isnan(vals)

        if isn.any():
            # Posición del primer NaN y valor inmediatamente anterior
            first_nan_pos = np.where(isn)[0][0]
            last_valid_value = vals[first_nan_pos - 1] if first_nan_pos > 0 else np.nan
        else:
            # Sin NaNs en la ventana: tomar el último valor válido
            last_valid_value = vals[-1] if vals.size > 0 else np.nan

        out.at[idx, 'atr_PnL'] = last_valid_value

    return out

def st_exit_date(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['st_Exit_Date'] = pd.NaT

    long_entries = df[df['Open_Trade'] == 1]
    short_entries = df[df['Open_Trade'] == -1]
    short_signals = df['Date'][df['Open_Trade'] == -1]
    long_signals = df['Date'][df['Open_Trade'] == 1]

    for index, row in long_entries.iterrows():
        next_short_signal_date = short_signals[short_signals > row['Date']]
        if not next_short_signal_date.empty:
            exit_index = df[df['Date'] == next_short_signal_date.iloc[0]].index
            if not exit_index.empty:
                df.loc[index, 'st_Exit_Date'] = df.loc[exit_index[0], 'Date']

    for index, row in short_entries.iterrows():
        next_long_signal_date = long_signals[long_signals > row['Date']]
        if not next_long_signal_date.empty:
            exit_index = df[df['Date'] == next_long_signal_date.iloc[0]].index
            if not exit_index.empty:
                df.loc[index, 'st_Exit_Date'] = df.loc[exit_index[0], 'Date']
    return df


def atr_exit_date(df: pd.DataFrame) -> pd.DataFrame:

    out = df.copy()
    out['Date']        = pd.to_datetime(out['Date'])
    out['Entry_Date']  = pd.to_datetime(out['Entry_Date'])
    out['st_Exit_Date'] = pd.to_datetime(out['st_Exit_Date'])

    out['atr_Exit_Date'] = pd.NaT
    out['atr_Duration']  = np.nan

    trade_mask = out['Type'].isin(['Long', 'Short'])

    for idx, trade in out.loc[trade_mask].iterrows():
        entry = trade['Entry_Date']
        final_exit = trade['st_Exit_Date']

        if pd.isna(entry) or pd.isna(final_exit):
            continue

        window_mask = (out['Date'] >= entry) & (out['Date'] <= final_exit)
        window_df = out.loc[window_mask].copy()
        first_nan_idx_in_window = window_df['atr_dyn'].isna().idxmax()

        if pd.notna(first_nan_idx_in_window) and first_nan_idx_in_window > window_df.index[0]:

            exit_idx_in_original_df = first_nan_idx_in_window - 1
            exit_date = out.loc[exit_idx_in_original_df, 'Date']

            out.at[idx, 'atr_Exit_Date'] = exit_date
            out.at[idx, 'atr_Duration'] = (exit_date - entry).total_seconds() / 60.0
        elif pd.isna(first_nan_idx_in_window) and not window_df.empty:
             # If no NaN is found, the exit is the end of the window (st_Exit_Date)
             out.at[idx, 'atr_Exit_Date'] = final_exit
             out.at[idx, 'atr_Duration'] = (final_exit - entry).total_seconds() / 60.0


    return out

def atr_exit_date(df: pd.DataFrame) -> pd.DataFrame:
    """
    Asigna atr_Exit_Date y atr_Duration tomando la fecha de la fila donde se determinó atr_PnL.
    Requiere que 'atr_PnL' exista (ejecutar antes atr_PnL()).
    """
    if 'atr_PnL' not in df.columns:
        raise ValueError("Debe ejecutar primero atr_PnL() para crear la columna 'atr_PnL'.")

    out = df.copy()
    out['Date']        = pd.to_datetime(out['Date'])
    out['Entry_Date']  = pd.to_datetime(out['Entry_Date'])
    out['st_Exit_Date'] = pd.to_datetime(out['st_Exit_Date'])

    out['atr_Exit_Date'] = pd.NaT
    out['atr_Duration']  = np.nan

    trade_mask = out['Type'].isin(['Long', 'Short'])

    for idx, trade in out.loc[trade_mask].iterrows():
        entry = trade['Entry_Date']
        final_exit = trade['st_Exit_Date']
        atr_val = trade['atr_PnL']

        if pd.isna(entry) or pd.isna(final_exit) or pd.isna(atr_val):
            continue

        window = out[(out['Date'] >= entry) & (out['Date'] <= final_exit)]
        if window.empty:
            continue

        # Filas donde el atr_dyn coincide con el valor atr_PnL asignado
        candidates = window[window['atr_dyn'] == atr_val]
        if candidates.empty:
            continue

        # Segunda aparición si existe; si no, la primera
        target = candidates.iloc[1] if len(candidates) > 1 else candidates.iloc[0]
        exit_date = target['Date']

        out.at[idx, 'atr_Exit_Date'] = exit_date
        out.at[idx, 'atr_Duration'] = (exit_date - entry).total_seconds() / 60.0

    return out

# ─────────────────────────────────────────────────────────────
# 3. SL_Duration  ─ minutos entre Entry_Date y SL_Exit_Date

def atr_Duration(df: pd.DataFrame) -> pd.DataFrame:
    if 'atr_Exit_Date' not in df.columns:
        raise ValueError("La columna 'atr_Exit_Date' no existe. Ejecuta SL_Exit_Date() antes.")

    out = df.copy()
    out['atr_Duration'] = np.nan

    mask = out['Type'].isin(['Long', 'Short'])
    durations = (out.loc[mask, 'atr_Exit_Date'] - out.loc[mask, 'Entry_Date']) \
                  .dt.total_seconds() / 60.0
    out.loc[mask, 'atr_Duration'] = durations

    return out

In [56]:
def atr_max_min(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df['Entry_Date'] = pd.to_datetime(df['Entry_Date'])
    df['Exit_Date']  = pd.to_datetime(df['st_Exit_Date'])

    df['ATR_Max'] = np.nan
    df['ATR_Min'] = np.nan

    opens = df[df['Type'].notna()]
    for idx, trade in opens.iterrows():
        entry   = trade['Entry_Date']
        exit_dt = trade['Exit_Date']
        exit_rows = df.index[df['Date'] == exit_dt]
        if not exit_rows.empty and exit_rows[0] > 0:
            prev_date = df.at[exit_rows[0] - 1, 'Date']
        else:
            continue

        mask = (df['Date'] >= entry) & (df['Date'] <= prev_date)
        vals = df.loc[mask, 'atr_mult'].dropna()
        if not vals.empty:
            df.at[idx, 'ATR_Max'] = vals.max()
            df.at[idx, 'ATR_Min'] = vals.min()

    return df

In [57]:

def atr_PnL_dollar(df: pd.DataFrame,
                   contract_multiplier: float = 100.0,  # XAUUSD típico: 1 lote = 100 oz
                   lots: float = 1.0,
                   lot_col: str | None = None,
                   multiplier_col: str | None = None) -> pd.DataFrame:
    """
    Convierte atr_PnL (múltiplos de ATR) a dólares por trade:
        atr_PnL_dollar = atr_PnL * ATR * contract_multiplier * lots

    Parámetros:
        contract_multiplier: valor por 1.0 de movimiento de precio por lote (p. ej. XAUUSD=100).
        lots: tamaño de lote por defecto si no hay columna.
        lot_col: nombre de columna con el tamaño de lote por fila (opcional).
        multiplier_col: nombre de columna con el multiplicador por fila (opcional).

    Requisitos:
        Columnas 'ATR' y 'atr_PnL' no nulas para el cálculo.
    """
    out = df.copy()
    out['atr_PnL_dollar'] = np.nan

    # Series de lots y multiplier (por fila si hay columnas, si no constantes)
    lots_series = out[lot_col].astype(float) if lot_col and lot_col in out.columns else lots
    mult_series = out[multiplier_col].astype(float) if multiplier_col and multiplier_col in out.columns else contract_multiplier

    mask = out['Type'].isin(['Long', 'Short']) & out['ATR'].notna() & out['atr_PnL'].notna()

    # Asegurar tipos numéricos
    atr   = pd.to_numeric(out.loc[mask, 'ATR'], errors='coerce')
    pnl_m = pd.to_numeric(out.loc[mask, 'atr_PnL'], errors='coerce')

    # Broadcasting correcto si lots/mult son escalares o series
    lts = lots_series if np.isscalar(lots_series) else lots_series.loc[mask]
    mlt = mult_series if np.isscalar(mult_series) else mult_series.loc[mask]

    out.loc[mask, 'atr_PnL_dollar'] = pnl_m * atr * mlt * lts

    return out


In [58]:
def atr_trade_analytics_table(df: pd.DataFrame):
    """
    Analiza resultados de trades ATR con formatos visuales:
      • Total PnL, Avg Profit y Avg Loss → $ con separador de miles, 0 decimales
      • Win (%) y Loss (%) → 2 decimales + símbolo %
      • Profit Factor → 2 decimales
      • # Days y conteos → separador de miles
    Usa la columna 'atr_PnL_$' (creada por atr_PnL_dollar()).
    """
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])

    analytics = pd.DataFrame(index =['ATR_Dyn'])
    analytics['# Days']            = (df['Date'].max() - df['Date'].min()).days
    analytics['Total PnL']         = round(df['atr_PnL_dollar'].sum(), 0)
    analytics['Total Trades']      = df['atr_PnL_dollar'].count()
    analytics['Number of Winners'] = (df['atr_PnL_dollar'] > 0).sum()
    analytics['Number of Losers']  = (df['atr_PnL_dollar'] < 0).sum()
    analytics['Win (%)']  = round(100 * analytics.at['ATR_Dyn', 'Number of Winners'] /
                                  analytics.at['ATR_Dyn', 'Total Trades'], 2)
    analytics['Loss (%)'] = round(100 * analytics.at['ATR_Dyn', 'Number of Losers']  /
                                  analytics.at['ATR_Dyn', 'Total Trades'], 2)
    analytics['Average Profit of Winning Trade'] = round(df.loc[df['atr_PnL_dollar'] > 0, 'atr_PnL_dollar'].mean(), 0)
    analytics['Average Loss of Losing Trade']    = round(abs(df.loc[df['atr_PnL_dollar'] < 0, 'atr_PnL_dollar'].mean()), 0)

    if {'Entry_Date','atr_Exit_Date'}.issubset(df.columns):
        df['Entry_Date']    = pd.to_datetime(df['Entry_Date'])
        df['atr_Exit_Date'] = pd.to_datetime(df['atr_Exit_Date'])
        analytics['Average Holding Time'] = (df['atr_Exit_Date'] - df['Entry_Date']).mean()

    tot_profit   = df.loc[df['atr_PnL_dollar'] > 0, 'atr_PnL_dollar'].sum()
    tot_loss_abs = abs(df.loc[df['atr_PnL_dollar'] < 0, 'atr_PnL_dollar'].sum())
    analytics['Profit Factor'] = round(tot_profit / tot_loss_abs, 2) if tot_loss_abs else np.nan

    def _fmt(val, metric):
        if metric in ('Win (%)', 'Loss (%)'):
            return f'{val:,.2f}%'
        if metric in ('Total PnL',
                      'Average Profit of Winning Trade',
                      'Average Loss of Losing Trade'):
            return f'${val:,.0f}'
        if metric in ('# Days', 'Total Trades', 'Number of Winners', 'Number of Losers'):
            return f'{val:,.0f}'
        if metric in ('Profit Factor',):
            return f'{val:,.2f}'
        return val

    styled = analytics.T.copy()
    styled['ATR_Dyn'] = [_fmt(v, idx) for idx, v in styled['ATR_Dyn'].items()]

    return (styled.style
                  .set_caption('')
                  .set_table_styles([
                      {'selector': 'th',  'props': [('font-weight','bold'), ('text-align','center')]},
                      {'selector': 'td',  'props': [('text-align','center')]},
                      {'selector': 'thead th', 'props': [('text-align','center')]}
                  ]))

In [59]:
def atr_dyn(data: pd.DataFrame, initial_atr: float = -2, first_step_atr: float = 1,
            gap_first_step_atr: float = 0.5) -> pd.DataFrame:
    df = data.copy()
    col_name = 'atr_dyn'
    df[col_name] = np.nan

    in_trade       = False
    trade_active   = False
    broken         = False
    sl_val         = initial_atr
    next_threshold = first_step_atr

    for idx, row in df.iterrows():

        if pd.notna(row['Entry_Date']):
            in_trade       = True
            trade_active   = True
            broken         = False
            sl_val         = initial_atr
            next_threshold = first_step_atr
            prev_sl        = sl_val

        if in_trade:

            best_pnl = np.nanmax([
                row['atr_mult_high'],
                row['atr_mult_low'],
                row['atr_mult_close']
            ])
            best_pnl = 0 if np.isnan(best_pnl) else best_pnl

            if trade_active and not broken:
                # subir stop en cada cruce de umbral
                while best_pnl >= next_threshold:
                    sl_val         += gap_first_step_atr
                    next_threshold += gap_first_step_atr

                # si cualquiera de los ATR-multiples cae bajo el stop previo…
                if (row['atr_mult_high']  < prev_sl
                    or row['atr_mult_low']  < prev_sl
                    or row['atr_mult_close']< prev_sl):
                    broken       = True
                    trade_active = False

            # 4) asignar NaN desde el quiebre hasta SL_Exit_Date
            df.at[idx, col_name] = np.nan if broken else sl_val
            prev_sl = sl_val

            # 5) cerrar el loop al llegar a la salida real
            if pd.notna(row['st_Exit_Date']) and row['Date'] == row['st_Exit_Date']:
                in_trade     = False
                trade_active = False

    return df

## Test

In [60]:
data = atr_multiples(data).round(4)
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type',
       'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low',
       'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL',
       'st_atr_max_PnL', 'atr_mult_close', 'atr_mult_high', 'atr_mult_low'],
      dtype='object')

In [61]:
initial_atr        = -1
first_step_atr     =  0.5
gap_first_step_atr =  2

data = atr_dyn(data, initial_atr, first_step_atr, gap_first_step_atr)
data = atr_PnL(data)
data = atr_exit_date(data)
data = atr_PnL_dollar(data)
data = atr_Duration(data)

In [62]:
atr_columns = ['Date', 'Close','ATR', 'Type', 'Entry_Date','Open_Trade','atr_Exit_Date', 'atr_mult_close', 'atr_mult_high', 'atr_mult_low',
               'atr_dyn', 'atr_PnL', 'atr_PnL_dollar']


#time_a, time_b = '2025-05-16 12:10:00', '2025-05-16 13:10:00'
time_a, time_b = '2025-05-15 14:50:00', '2025-05-15 18:00:00'

data.loc[data['Open_Trade'].notna(), atr_columns].head(20)
data.loc[(data['Date']>= time_a) & (data['Date']<= time_b), atr_columns]

,Date,Close,ATR,Type,Entry_Date,Open_Trade,atr_Exit_Date,atr_mult_close,atr_mult_high,atr_mult_low,atr_dyn,atr_PnL,atr_PnL_dollar
5922,2025-05-15 14:50:00,3178.09,3.3307,Long,2025-05-15 14:50:00,1.0,2025-05-15 15:00:00,-0.4984,0.2342,-0.4984,-1.0,1.0,333.07
5923,2025-05-15 14:55:00,3179.18,3.2342,NaN,NaT,NaN,NaT,-0.2342,0.3603,-0.2342,-1.0,NaN,NaN
5924,2025-05-15 15:00:00,3181.54,3.3346,NaN,NaT,NaN,NaT,-0.1711,1.2220,-0.1711,1.0,NaN,NaN
5925,2025-05-15 15:05:00,3182.12,3.2207,NaN,NaT,NaN,NaT,0.8527,1.3751,0.8527,NaN,NaN,NaN
5926,2025-05-15 15:10:00,3180.39,3.2350,NaN,NaT,NaN,NaT,0.2132,1.2400,0.2132,NaN,NaN,NaN
5927,2025-05-15 15:15:00,3179.38,3.2375,NaN,NaT,NaN,NaT,-0.0360,0.9457,-0.0360,NaN,NaN,NaN
5928,2025-05-15 15:20:00,3183.09,3.5977,NaN,NaT,NaN,NaT,-0.0751,2.4109,-0.0751,NaN,NaN,NaN
5929,2025-05-15 15:25:00,3182.69,3.5307,NaN,NaT,NaN,NaT,0.7146,1.5132,0.7146,NaN,NaN,NaN
5930,2025-05-15 15:30:00,3186.11,3.6735,NaN,NaT,NaN,NaT,1.0598,2.7201,1.0598,NaN,NaN,NaN
5931,2025-05-15 15:35:00,3182.77,3.7468,NaN,NaT,NaN,NaT,1.3421,2.7532,1.3421,NaN,NaN,NaN


In [63]:
data_atr_pnl = data.copy()
data_atr_pnl = data_atr_pnl[data_atr_pnl['Entry_Date'].notna()]
#data_atr_pnl.columns

In [64]:
trade_analysis = atr_trade_analytics_table(data_atr_pnl)
trade_analysis

,ATR_Dyn
# Days,101
Total PnL,"$212,603"
Total Trades,"1,852"
Number of Winners,"1,222"
Number of Losers,630
Win (%),65.98%
Loss (%),34.02%
Average Profit of Winning Trade,$326
Average Loss of Losing Trade,$296
Average Holding Time,0 days 00:18:32.203023758


In [65]:
print(data_atr_pnl['atr_PnL_dollar'].sum())

212602.66


## Genetic

In [ ]:

def atr_gen_fit_function(data: pd.DataFrame,
                         gen_atr_initial: float,
                         gen_atr_first:   float,
                         gen_atr_gap:     float) -> pd.DataFrame:
    df = atr_dyn(data.copy(),
                 initial_atr=gen_atr_initial,
                 first_step_atr=gen_atr_first,
                 gap_first_step_atr=gen_atr_gap)

    df = atr_PnL(df)
    df = atr_exit_date(df)
    df = atr_PnL_dollar(df)
    total_profit = df['atr_PnL_dollar'].sum()

    summary = pd.DataFrame({
        'gen_atr_initial': [gen_atr_initial],
        'gen_atr_first':   [gen_atr_first],
        'gen_atr_gap':     [gen_atr_gap],
        'total_profit':    [total_profit]
    })

    return summary


In [ ]:
# ───────────────────────── CONFIG ─────────────────────────
GEN_ATR_INITIAL_RANGE = (-5.0, -1.0, 0.1)
GEN_FIRST_STEP        = ( 0.5, 5.0, 0.1)
GEN_GAP_STEP          = ( 0.1, 2.0, 0.1)

# ───────────────────── FUNCIONES AUX ──────────────────────
def _discrete_space(low: float, high: float, step: float) -> np.ndarray:
    n = int(round((high - low) / step)) + 1
    return np.round(low + np.arange(n) * step, 10)

_SPACE_INIT  = _discrete_space(*GEN_ATR_INITIAL_RANGE)
_SPACE_FIRST = _discrete_space(*GEN_FIRST_STEP)
_SPACE_GAP   = _discrete_space(*GEN_GAP_STEP)

def _random_gene() -> Tuple[float, float, float]:
    return (np.random.choice(_SPACE_INIT),
            np.random.choice(_SPACE_FIRST),
            np.random.choice(_SPACE_GAP))

def _mutate_gene(gene: Tuple[float, float, float],
                 p_mut: float,
                 mut_span: int = 1) -> Tuple[float, float, float]:
    gi, gf, gg = gene
    if np.random.rand() < p_mut:
        gi = _SPACE_INIT[np.clip(np.where(_SPACE_INIT == gi)[0][0] +
                                 np.random.randint(-mut_span, mut_span+1),
                                 0, len(_SPACE_INIT)-1)]
    if np.random.rand() < p_mut:
        gf = _SPACE_FIRST[np.clip(np.where(_SPACE_FIRST == gf)[0][0] +
                                  np.random.randint(-mut_span, mut_span+1),
                                  0, len(_SPACE_FIRST)-1)]
    if np.random.rand() < p_mut:
        gg = _SPACE_GAP[np.clip(np.where(_SPACE_GAP == gg)[0][0] +
                                np.random.randint(-mut_span, mut_span+1),
                                0, len(_SPACE_GAP)-1)]
    return (gi, gf, gg)

def _crossover_gene(p1: Tuple[float, float, float],
                    p2: Tuple[float, float, float]) -> Tuple[float, float, float]:
    mask = np.random.randint(0, 2, 3).astype(bool)
    return tuple(np.where(mask, p1, p2))

# ─────────── tqdm helper ───────────
try:
    from tqdm.auto import trange, tqdm
    _HAS_TQDM = True
except Exception:
    _HAS_TQDM = False

# ───────────────────── GA PRINCIPAL ────────────────────────
def run_ga_atr(
    data: pd.DataFrame,
    pop_size: int = 30,
    generations: int = 40,
    elitism: int = 2,
    p_crossover: float = 0.8,
    p_mutation: float = 0.2,
    tournament_k: int = 3,
    random_state: int | None = None,
    show_progress: bool = True,
    progress_every: int = 1,
    callback: Callable[[int, Tuple[float,float,float], float, List[Dict[str,Any]]], None] | None = None
) -> Tuple[Dict[str, float], pd.DataFrame]:

    if random_state is not None:
        np.random.seed(random_state)

    fitness_cache: Dict[Tuple[float, float, float], float] = {}

    def evaluate(gene: Tuple[float, float, float]) -> float:
        if gene in fitness_cache:
            return fitness_cache[gene]
        res = atr_gen_fit_function(data, *gene)
        fit = float(res['total_profit'].iloc[0])
        fitness_cache[gene] = fit
        return fit

    population: List[Tuple[float, float, float]] = [_random_gene() for _ in range(pop_size)]
    results_records: List[Dict[str, Any]] = []

    def tournament_select() -> Tuple[float, float, float]:
        idxs = np.random.choice(len(population), tournament_k, replace=False)
        contenders = [population[i] for i in idxs]
        fits = [evaluate(c) for c in contenders]
        return contenders[int(np.argmax(fits))]

    # Iterator de generaciones
    gen_iter = trange(generations, desc="GA ATR", leave=False) if (_HAS_TQDM and show_progress) else range(generations)

    for gen in gen_iter:
        # Barra interna opcional para individuos
        if _HAS_TQDM and show_progress:
            indiv_iter = tqdm(population, desc=f"Gen {gen}", leave=False)
        else:
            indiv_iter = population

        # evaluar población
        for ind in indiv_iter:
            fit = evaluate(ind)
            results_records.append({
                'generation':      gen,
                'gen_atr_initial': ind[0],
                'gen_atr_first':   ind[1],
                'gen_atr_gap':     ind[2],
                'total_profit':    fit
            })

        ranked   = sorted(population, key=evaluate, reverse=True)
        best_gene = ranked[0]
        best_fit  = evaluate(best_gene)

        # progreso externo
        if _HAS_TQDM and show_progress:
            gen_iter.set_postfix(best_fit=f"{best_fit:,.2f}")
        elif show_progress and (gen % progress_every == 0):
            print(f"[Gen {gen:03d}] best_fit={best_fit:,.2f}  gene={best_gene}", flush=True)

        if callback is not None:
            callback(gen, best_gene, best_fit, results_records)

        # Reproducción
        next_pop = ranked[:elitism]
        while len(next_pop) < pop_size:
            parent1 = tournament_select()
            parent2 = tournament_select()
            child = parent1
            if np.random.rand() < p_crossover:
                child = _crossover_gene(parent1, parent2)
            child = _mutate_gene(child, p_mutation)
            next_pop.append(child)

        population = next_pop

    # final
    best = max(population, key=evaluate)
    best_fit = evaluate(best)
    best_params = {
        'gen_atr_initial': best[0],
        'gen_atr_first':   best[1],
        'gen_atr_gap':     best[2],
        'total_profit':    best_fit
    }

    results_df = (pd.DataFrame(results_records)
                    .drop_duplicates(subset=['gen_atr_initial','gen_atr_first','gen_atr_gap','generation'], keep='last')
                    .reset_index(drop=True))

    return best_params, results_df

In [ ]:
best_params, results_atr = run_ga_atr(data, pop_size=40, generations=50, random_state=42)
results_atr.to_csv(root_data + 'Results/'+symbol+'ATR_Genetic_Results_M5.csv')
print(best_params)
print(results_atr.tail())

In [ ]:
initial_atr = best_params['gen_atr_initial']
first_step_atr = best_params['gen_atr_first']
gap_first_step_atr = best_params['gen_atr_gap']

data = atr_dyn(data, initial_atr, first_step_atr, gap_first_step_atr)
data = atr_PnL(data)
data = atr_exit_date(data)
data = atr_PnL_dollar(data)
data = atr_Duration(data)
data[f'atr_PnL_dollar_{initial_atr}_{first_step_atr}_{gap_first_step_atr}'] = data['atr_PnL_dollar'].copy()


In [ ]:
data_atr_pnl = data.copy()
data_atr_pnl = data_atr_pnl[data_atr_pnl['Entry_Date'].notna()]
data_atr_pnl.columns

In [ ]:
trade_analysis = atr_trade_analytics_table(data_atr_pnl)
trade_analysis

In [ ]:
atr_columns = ['Date', 'Close','ATR', 'Type', 'Entry_Date', 'atr_mult_close', 'atr_mult_high', 'atr_mult_low', 'atr_dyn', 'atr_dyn', 'atr_PnL', 'atr_Exit_Date',
       'atr_Duration', 'atr_PnL_dollar']

#data.loc[data['Entry_Date']!=0, atr_columns]

#data.loc[(data['Date']>= '2019-07-08 03:10:00') & (data['Date']<= '2019-07-08 04:30:00'),  atr_columns]
#data.loc[(data['Date']>= '2019-07-08 02:30:00') & (data['Date']<= '2019-07-08 04:30:00'), atr_columns]

data.loc[(data['Date']>= '2019-08-26 05:05:00') & (data['Date']<= '2019-08-26 08:05:00'),columns_to_see] # Max 400 st_Max

In [ ]:
data.columns

In [ ]:
data.to_csv(root_data+'Results/'+symbol+'_'+time_frame+'_ATR_Labels.csv')

# ATR_HIGH

## Functions

In [66]:
def atr_multiples(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula tres métricas expresadas en múltiplos de ATR para cada barra
    comprendida entre la apertura (Entry_Date) y la penúltima barra antes del
    cierre (st_Exit_Date) de cada trade Long/Short.

    Se añaden/actualizan tres columnas:
        • atr_mult_close : usa el precio low (misma lógica del original “close”)
        • atr_mult_high  : usa high
        • atr_mult_low   : usa low

    El DataFrame se modifica in-place y se devuelve por conveniencia.
    """
    # ── asegurar tipos datetime --------------------------------------------
    for col in ('Date', 'Entry_Date', 'st_Exit_Date'):
        df[col] = pd.to_datetime(df[col])

    # ── inicializar columnas destino ---------------------------------------
    for col in ('atr_mult_close', 'atr_mult_high', 'atr_mult_low'):
        if col not in df.columns:
            df[col] = np.nan
        else:
            df[col].values[:] = np.nan   # limpiar si ya existían

    # ── iterar sobre cada trade abierto ------------------------------------
    trades = df[df['Type'].isin(['Long', 'Short'])]
    for _, trade in trades.iterrows():
        entry_dt = trade['Entry_Date']
        exit_dt  = trade['st_Exit_Date']
        base_px  = trade['Close']
        atr_base = trade['ATR']
        is_long  = trade['Type'] == 'Long'

        # fila de salida y fecha previa
        exit_idx = df.index[df['Date'] == exit_dt]
        if exit_idx.empty or exit_idx[0] == 0:
            continue                    # no hay barra previa para medir
        prev_date = df.at[exit_idx[0] - 1, 'Date']

        # máscara de barras a rellenar
        mask = (df['Date'] >= entry_dt) & (df['Date'] <= prev_date)

        # --- close (usa low como en función original) ----------------------
        if is_long:
            df.loc[mask, 'atr_mult_close'] = (df.loc[mask, 'Low']  - base_px) / atr_base
            df.loc[mask, 'atr_mult_high']  = (df.loc[mask, 'High'] - base_px) / atr_base
            df.loc[mask, 'atr_mult_low']   = (df.loc[mask, 'Low']  - base_px) / atr_base
        else:  # Short
            df.loc[mask, 'atr_mult_close'] = (base_px - df.loc[mask, 'Low'])  / atr_base
            df.loc[mask, 'atr_mult_high']  = (base_px - df.loc[mask, 'High']) / atr_base
            df.loc[mask, 'atr_mult_low']   = (base_px - df.loc[mask, 'Low'])  / atr_base

    return df

In [67]:
def atr_H_PnL(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['atr_H_PnL'] = np.nan

    entries = out[out['Type'].isin(['Long', 'Short'])]

    for idx, row in entries.iterrows():
        m = (out['Date'] >= row['Entry_Date']) & (out['Date'] <= row['st_Exit_Date'])
        out.at[idx, 'atr_H_PnL'] = out.loc[m, 'atr_H_dyn'].max()

    return out

def atr_H_exit_date(df: pd.DataFrame) -> pd.DataFrame:
    """
    Asigna atr_Exit_Date y atr_Duration tomando la fecha de la fila donde se determinó atr_H_PnL.
    Requiere que 'atr_H_PnL' exista (ejecutar antes atr_H_PnL()).
    """
    if 'atr_H_PnL' not in df.columns:
        raise ValueError("Debe ejecutar primero atr_PnL() para crear la columna 'atr_H_PnL'.")

    out = df.copy()
    out['Date']        = pd.to_datetime(out['Date'])
    out['Entry_Date']  = pd.to_datetime(out['Entry_Date'])
    out['st_Exit_Date'] = pd.to_datetime(out['st_Exit_Date'])

    out['atr_H_Exit_Date'] = pd.NaT
    out['atr_H_Duration']  = np.nan

    trade_mask = out['Type'].isin(['Long', 'Short'])

    for idx, trade in out.loc[trade_mask].iterrows():
        entry = trade['Entry_Date']
        final_exit = trade['st_Exit_Date']
        atr_val = trade['atr_H_PnL']

        if pd.isna(entry) or pd.isna(final_exit) or pd.isna(atr_val):
            continue

        window = out[(out['Date'] >= entry) & (out['Date'] <= final_exit)]
        if window.empty:
            continue

        # Filas donde el atr_dyn coincide con el valor atr_PnL asignado
        candidates = window[window['atr_H_dyn'] == atr_val]
        if candidates.empty:
            continue

        # Segunda aparición si existe; si no, la primera
        target = candidates.iloc[1] if len(candidates) > 1 else candidates.iloc[0]
        exit_date = target['Date']

        out.at[idx, 'atr_H_Exit_Date'] = exit_date
        out.at[idx, 'atr_H_Duration'] = (exit_date - entry).total_seconds() / 60.0

    return out

# ─────────────────────────────────────────────────────────────
# 3. SL_Duration  ─ minutos entre Entry_Date y SL_Exit_Date
# ─────────────────────────────────────────────────────────────
def atr_H_Duration(df: pd.DataFrame) -> pd.DataFrame:
    if 'atr_H_Exit_Date' not in df.columns:
        raise ValueError("La columna 'atr_Exit_Date' no existe. Ejecuta SL_Exit_Date() antes.")

    out = df.copy()
    out['atr_H_Duration'] = np.nan

    mask = out['Type'].isin(['Long', 'Short'])
    durations = (out.loc[mask, 'atr_H_Exit_Date'] - out.loc[mask, 'Entry_Date']) \
                  .dt.total_seconds() / 60.0
    out.loc[mask, 'atr_H_Duration'] = durations

    return out

In [68]:
def atr_H_max_min(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df['Entry_Date'] = pd.to_datetime(df['Entry_Date'])
    df['Exit_Date']  = pd.to_datetime(df['atr_H_Exit_Date'])

    df['ATR_Max'] = np.nan
    df['ATR_Min'] = np.nan

    opens = df[df['Type'].notna()]
    for idx, trade in opens.iterrows():
        entry   = trade['Entry_Date']
        exit_dt = trade['Exit_Date']
        exit_rows = df.index[df['Date'] == exit_dt]
        if not exit_rows.empty and exit_rows[0] > 0:
            prev_date = df.at[exit_rows[0] - 1, 'Date']
        else:
            continue

        mask = (df['Date'] >= entry) & (df['Date'] <= prev_date)
        vals = df.loc[mask, 'atr_H_mult'].dropna()
        if not vals.empty:
            df.at[idx, 'ATR_H_Max'] = vals.max()
            df.at[idx, 'ATR_H_Min'] = vals.min()

    return df

In [69]:
def atr_H_PnL_dollar(data: pd.DataFrame) -> pd.DataFrame:

    df = data.copy()
    df['atr_H_PnL_dollar'] = np.nan

    mask = df['Type'].isin(['Long', 'Short']) & df['ATR'].notna() & df['atr_H_PnL'].notna()
    df.loc[mask, 'atr_H_PnL_dollar'] = (df.loc[mask, 'ATR'] * df.loc[mask, 'atr_H_PnL']) * 100
    df['atr_H_PnL_dollar'] = df['atr_H_PnL_dollar'].round(0) # Round to 0 decimal places


    return df

In [70]:
def atr_H_trade_analytics_table(df: pd.DataFrame):
    """
    Analiza resultados de trades ATR con formatos visuales:
      • Total PnL, Avg Profit y Avg Loss → $ con separador de miles, 0 decimales
      • Win (%) y Loss (%) → 2 decimales + símbolo %
      • Profit Factor → 2 decimales
      • # Days y conteos → separador de miles
    Usa la columna 'atr_PnL_$' (creada por atr_PnL_dollar()).
    """
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])

    analytics = pd.DataFrame(index =['atr_H_dyn'])
    analytics['# Days']            = (df['Date'].max() - df['Date'].min()).days
    analytics['Total PnL']         =  round(df['atr_H_PnL_dollar'].sum(), 0)
    analytics['Total Trades']      =  df['atr_H_PnL_dollar'].count()
    analytics['Number of Winners'] = (df['atr_H_PnL_dollar'] > 0).sum()
    analytics['Number of Losers']  = (df['atr_H_PnL_dollar'] < 0).sum()
    analytics['Win (%)']  = round(100 * analytics.at['atr_H_dyn', 'Number of Winners'] /
                                  analytics.at['atr_H_dyn', 'Total Trades'], 2)
    analytics['Loss (%)'] = round(100 * analytics.at['atr_H_dyn', 'Number of Losers']  /
                                  analytics.at['atr_H_dyn', 'Total Trades'], 2)
    analytics['Average Profit of Winning Trade'] = round(df.loc[df['atr_H_PnL_dollar'] > 0, 'atr_H_PnL_dollar'].mean(), 0)
    analytics['Average Loss of Losing Trade']    = round(abs(df.loc[df['atr_H_PnL_dollar'] < 0, 'atr_H_PnL_dollar'].mean()), 0)

    if {'Entry_Date','atr_Exit_Date'}.issubset(df.columns):
        df['Entry_Date']    = pd.to_datetime(df['Entry_Date'])
        df['atr_Exit_Date'] = pd.to_datetime(df['atr_H_Exit_Date'])
        analytics['Average Holding Time'] = (df['atr_H_Exit_Date'] - df['Entry_Date']).mean()

    tot_profit   = df.loc[df['atr_H_PnL_dollar'] > 0, 'atr_H_PnL_dollar'].sum()
    tot_loss_abs = abs(df.loc[df['atr_H_PnL_dollar'] < 0, 'atr_H_PnL_dollar'].sum())
    analytics['Profit Factor'] = round(tot_profit / tot_loss_abs, 2) if tot_loss_abs else np.nan

    def _fmt(val, metric):
        if metric in ('Win (%)', 'Loss (%)'):
            return f'{val:,.2f}%'
        if metric in ('Total PnL',
                      'Average Profit of Winning Trade',
                      'Average Loss of Losing Trade'):
            return f'${val:,.0f}'
        if metric in ('# Days', 'Total Trades', 'Number of Winners', 'Number of Losers'):
            return f'{val:,.0f}'
        if metric in ('Profit Factor',):
            return f'{val:,.2f}'
        return val

    styled = analytics.T.copy()
    styled['atr_H_dyn'] = [_fmt(v, idx) for idx, v in styled['atr_H_dyn'].items()]

    return (styled.style
                  .set_caption('')
                  .set_table_styles([
                      {'selector': 'th',  'props': [('font-weight','bold'), ('text-align','center')]},
                      {'selector': 'td',  'props': [('text-align','center')]},
                      {'selector': 'thead th', 'props': [('text-align','center')]}
                  ]))

## Test

In [71]:
data = atr_multiples(data).round(4)
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type',
       'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low',
       'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL',
       'st_atr_max_PnL', 'atr_mult_close', 'atr_mult_high', 'atr_mult_low',
       'atr_dyn', 'atr_PnL', 'atr_Exit_Date', 'atr_Duration',
       'atr_PnL_dollar'],
      dtype='object')

In [72]:
def atr_H_dyn(
    data: pd.DataFrame,
    initial_H_atr: float = -2.0,
    gap_H_atr: float = 1.0
) -> pd.DataFrame:
    """
    Calcula un trailing‑stop en múltiplos de ATR y lo guarda como 'atr_H_dyn'.

    • La primera barra del trade fija el stop en `initial_H_atr`.
    • A partir de la barra siguiente:
         stop = max(stop_anterior, best_pnl – gap_H_atr)
    • Si cualquiera de los múltiplos (high/low/close) cruza el stop anterior,
      se considera "roto" y devuelve NaN hasta el cierre oficial.
    """
    df = data.copy()
    col_name = "atr_H_dyn"
    df[col_name] = np.nan

    in_trade     = False
    trade_active = False
    broken       = False
    sl_val       = initial_H_atr
    prev_sl      = initial_H_atr

    for idx, row in df.iterrows():

        # Si hay apertura de trade, inicializamos
        if pd.notna(row["Entry_Date"]):
            in_trade     = True
            trade_active = True
            broken       = False
            sl_val       = initial_H_atr
            prev_sl      = sl_val
            df.at[idx, col_name] = sl_val  # primer valor stoploss
            continue

        # Si no estamos en trade, seguimos
        if not in_trade:
            continue

        # Calculamos el mejor múltiplo de ATR de la barra
        best_pnl = np.nanmax([
            row["atr_mult_high"],
            row["atr_mult_low"],
            row["atr_mult_close"]
        ])
        best_pnl = 0 if np.isnan(best_pnl) else best_pnl

        # Actualizamos el stoploss solo si está activo
        if trade_active and not broken:
            candidate = best_pnl - gap_H_atr

            # Si el precio perfora el stop anterior, rompemos el stoploss
            if (
                row["atr_mult_high"] < prev_sl or
                row["atr_mult_low"] < prev_sl or
                row["atr_mult_close"] < prev_sl
            ):
                broken       = True
                trade_active = False
                df.at[idx, col_name] = np.nan
            else:
                # Ajustamos el stoploss sin permitir retroceso
                sl_val = max(prev_sl, candidate)
                df.at[idx, col_name] = sl_val
                prev_sl = sl_val

        else:
            # Si está roto, mantenemos NaN
            df.at[idx, col_name] = np.nan

        # Cierre oficial del trade: reseteamos
        if pd.notna(row["st_Exit_Date"]) and row["Date"] == row["st_Exit_Date"]:
            in_trade     = False
            trade_active = False

    return df

In [73]:
initial_H_atr = -1.3
gap_H_atr     =  0.1

data = atr_H_dyn(data, initial_H_atr, gap_H_atr)
data = atr_H_PnL(data)
data = atr_H_exit_date(data)
data = atr_H_PnL_dollar(data)
data = atr_H_Duration(data)

In [74]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type',
       'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low',
       'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL',
       'st_atr_max_PnL', 'atr_mult_close', 'atr_mult_high', 'atr_mult_low',
       'atr_dyn', 'atr_PnL', 'atr_Exit_Date', 'atr_Duration', 'atr_PnL_dollar',
       'atr_H_dyn', 'atr_H_PnL', 'atr_H_Exit_Date', 'atr_H_Duration',
       'atr_H_PnL_dollar'],
      dtype='object')

In [ ]:
atr_H_columns = ['Date', 'Close','ATR', 'Type', 'Entry_Date', 'atr_H_Exit_Date', 'atr_mult_close', 'atr_mult_high', 'atr_mult_low',
               'atr_H_dyn', 'atr_H_PnL',  'atr_H_PnL_dollar']

#time_a, time_b = '2025-05-16 12:10:00', '2025-05-16 13:10:00'
time_a, time_b = '2025-05-15 14:50:00', '2025-05-15 18:00:00'

data.loc[data['Open_Trade'].notna(), atr_columns].head(20)
data.loc[(data['Date']>= time_a) & (data['Date']<= time_b), atr_H_columns]

In [76]:
data_H_atr_pnl = data.copy()
data_H_atr_pnl = data_H_atr_pnl[data_H_atr_pnl['Entry_Date'].notna()]
#data_atr_pnl.columns

In [77]:
trade_H_analysis = atr_H_trade_analytics_table(data_H_atr_pnl)
trade_H_analysis

,atr_H_dyn
# Days,101
Total PnL,"$327,469"
Total Trades,"2,807"
Number of Winners,"2,256"
Number of Losers,545
Win (%),80.37%
Loss (%),19.42%
Average Profit of Winning Trade,$197
Average Loss of Losing Trade,$213
Average Holding Time,0 days 00:20:06.626291414


In [78]:
print(data_H_atr_pnl['atr_H_PnL_dollar'].sum())

327469.0


## Genetic

In [ ]:
def atr_H_gen_fit_function(
        data: pd.DataFrame,
        initial_H_atr: float,
        gap_H_atr: float                       # ← nombre consistente
    ) -> pd.DataFrame:
    """
    Evalúa un par (initial_H_atr, gap_H_atr) y devuelve un DataFrame‑resumen
    con el beneficio total; se usa como función de fitness del GA.
    """

    # 1) Calcula la serie dinámica del stop
    df = atr_H_dyn(
        data.copy(),
        initial_H_atr=initial_H_atr,
        gap_H_atr=gap_H_atr                  # ← mismo nombre aquí
    )

    # 2) Métricas de rendimiento
    df = atr_H_PnL(df)
    df = atr_H_exit_date(df)
    df = atr_H_PnL_dollar(df)
    total_H_profit = df['atr_H_PnL_dollar'].sum()

    # 3) Resumen para el GA
    summary = pd.DataFrame({
        'gen_initial_H_atr': [initial_H_atr],
        'gen_atr_H_gap':     [gap_H_atr],
        'total_H_profit':    [total_H_profit]
    })

    return summary

In [ ]:
# ───────────────────────── CONFIG ─────────────────────────
GEN_INITIAL_H_ATR = (-5.0, -1.0, 0.1)
GEN_GAP_H_STEP    = ( 0.1, 2.0, 0.1)

# ───────────────────── FUNCIONES AUX ──────────────────────
def _discrete_space(low: float, high: float, step: float) -> np.ndarray:
    n = int(round((high - low) / step)) + 1
    return np.round(low + np.arange(n) * step, 10)

_SPACE_INIT  = _discrete_space(*GEN_INITIAL_H_ATR)
_SPACE_GAP   = _discrete_space(*GEN_GAP_H_STEP)

def _random_gene() -> Tuple[float, float]:
    return (np.random.choice(_SPACE_INIT),
            np.random.choice(_SPACE_GAP))

def _mutate_gene(gene: Tuple[float, float],
                 p_mut: float,
                 mut_span: int = 1) -> Tuple[float, float]:
    gi, gg = gene
    if np.random.rand() < p_mut:
        gi = _SPACE_INIT[np.clip(np.where(_SPACE_INIT == gi)[0][0] +
                                 np.random.randint(-mut_span, mut_span+1),
                                 0, len(_SPACE_INIT)-1)]

    if np.random.rand() < p_mut:
        gg = _SPACE_GAP[np.clip(np.where(_SPACE_GAP == gg)[0][0] +
                                np.random.randint(-mut_span, mut_span+1),
                                0, len(_SPACE_GAP)-1)]
    return (gi, gg)

def _crossover_gene(p1: Tuple[float, float],
                    p2: Tuple[float, float]) -> Tuple[float, float]:
    mask = np.random.randint(0, 2, 2).astype(bool) # Changed mask size to 2
    return tuple(np.where(mask, p1, p2))

# ─────────── tqdm helper ───────────
try:
    from tqdm.auto import trange, tqdm
    _HAS_TQDM = True
except Exception:
    _HAS_TQDM = False

# ───────────────────── GA PRINCIPAL ────────────────────────
def run_ga_atr_H(
    data: pd.DataFrame,
    pop_size: int = 30,
    generations: int = 40,
    elitism: int = 2,
    p_crossover: float = 0.8,
    p_mutation: float = 0.2,
    tournament_k: int = 3,
    random_state: int | None = None,
    show_progress: bool = True,
    progress_every: int = 1,
    callback: Callable[[int, Tuple[float,float], float, List[Dict[str,Any]]], None] | None = None
) -> Tuple[Dict[str, float], pd.DataFrame]:

    if random_state is not None:
        np.random.seed(random_state)

    fitness_cache: Dict[Tuple[float, float], float] = {}

    def evaluate(gene: Tuple[float, float]) -> float:
        if gene in fitness_cache:
            return fitness_cache[gene]
        # Call the correct fit function
        res = atr_H_gen_fit_function(data, *gene)
        fit = float(res['total_H_profit'].iloc[0])
        fitness_cache[gene] = fit
        return fit

    population: List[Tuple[float, float]] = [_random_gene() for _ in range(pop_size)]
    results_records: List[Dict[str, Any]] = []

    def tournament_select() -> Tuple[float, float]:
        idxs = np.random.choice(len(population), tournament_k, replace=False)
        contenders = [population[i] for i in idxs]
        fits = [evaluate(c) for c in contenders]
        return contenders[int(np.argmax(fits))]

    # Iterator de generaciones
    gen_iter = trange(generations, desc="GA ATR", leave=False) if (_HAS_TQDM and show_progress) else range(generations)

    for gen in gen_iter:
        # Barra interna opcional para individuos
        if _HAS_TQDM and show_progress:
            indiv_iter = tqdm(population, desc=f"Gen {gen}", leave=False)
        else:
            indiv_iter = population

        # evaluar población
        for ind in indiv_iter:
            fit = evaluate(ind)
            results_records.append({
                'generation': gen,
                'gen_initial_H_atr': ind[0],
                'gen_atr_H_gap': ind[1],
                'total_H_profit': fit
            })

        ranked   = sorted(population, key=evaluate, reverse=True)
        best_gene = ranked[0]
        best_fit  = evaluate(best_gene)

        # progreso externo
        if _HAS_TQDM and show_progress:
            gen_iter.set_postfix(best_fit=f"{best_fit:,.2f}")
        elif show_progress and (gen % progress_every == 0):
            print(f"[Gen {gen:03d}] best_fit={best_fit:,.2f}  gene={best_gene}", flush=True)

        if callback is not None:
            callback(gen, best_gene, best_fit, results_records)

        # Reproducción
        next_pop = ranked[:elitism]
        while len(next_pop) < pop_size:
            parent1 = tournament_select()
            parent2 = tournament_select()
            child = parent1
            if np.random.rand() < p_crossover:
                child = _crossover_gene(parent1, parent2)
            child = _mutate_gene(child, p_mutation)
            next_pop.append(child)

        population = next_pop

    # final
    best = max(population, key=evaluate)
    best_fit = evaluate(best)
    best_params = {
        'gen_atr_H_initial': best[0],
        'gen_atr_H_gap':     best[1],
        'total_H_profit':    best_fit
    }

    results_df = (pd.DataFrame(results_records)
                    .drop_duplicates(subset=['gen_initial_H_atr','gen_atr_H_gap','generation', 'total_H_profit'], keep='last')
                    .reset_index(drop=True))

    return best_params, results_df

In [ ]:
best_H_params, results_H_atr = run_ga_atr_H(data, pop_size=40, generations=50, random_state=42)
results_H_atr.to_csv(root_data + 'Results/'+symbol+'M5_ATR_H_Labels.csv')
print(best_H_params)
print(results_H_atr.tail())

In [ ]:
initial_H_atr = best_H_params['gen_atr_H_initial']
gap_step_H_atr = best_H_params['gen_atr_H_gap']

data = atr_H_dyn(data, initial_H_atr, gap_step_H_atr)
data = atr_H_PnL(data)
data = atr_H_exit_date(data)
data = atr_H_PnL_dollar(data)
data = atr_H_Duration(data)
data[f'atr_H_PnL_dollar_{initial_H_atr}_{gap_step_H_atr}'] = data['atr_H_PnL_dollar'].copy()


In [ ]:
data_atr_H_pnl = data.copy()
data_atr_H_pnl = data_atr_H_pnl[data_atr_H_pnl['Entry_Date'].notna()]
data_atr_H_pnl.columns

In [ ]:
trade_analysis_H = atr_H_trade_analytics_table(data_atr_H_pnl)
trade_analysis_H

In [ ]:
atr_H_columns = ['Date', 'Close','ATR', 'Type', 'Entry_Date', 'atr_mult_close', 'atr_mult_high', 'atr_mult_low',
               'atr_H_PnL', 'atr_H_Exit_Date', 'atr_H_Duration', 'atr_H_PnL_dollar']

#data.loc[((data['Entry_Date']!=0) & (data['atr_H_PnL_dollar'].notna())), atr_H_columns]

In [ ]:
data.columns

In [ ]:
data.to_csv(root_data+'Results/'+symbol+'_'+time_frame+'_ATR_HIGH_Labels.csv')

In [ ]:
data.loc[data['Open_Trade'].notna(),['Date', 'ATR', 'Open_Trade', 'trade type', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL', 'st_atr_max_PnL']]